# Drought Identification and Trend Analysis using Google Earth Engine (GEE) - Admin 2 Level

Author - Nitin Magima (edited by J. Sim for Rwanda)

Date - Oct 2024

Version - 1.0

[**Google Earth Engine**](https://earthengine.google.com/) is a public data archive of petabytes of historical satellite imagery and geospatial datasets. The advantage lies in its remarkable computation speed as processing is outsourced to Google servers. The platform provides a variety of constantly updated datasets; no download of raw imagery is required. While it is free of charge, one still needs to activate access to Google Earth Engine with a valid Google account.

The Jupyter Notebook uses the following [paper](https://www.mdpi.com/2071-1050/13/3/1042) to recreate the analysis for drought identification and trend analysis in Kenya using Google Earth Engine (GEE) for Python, follow these steps:

## Analysis Steps

### Step 1: Data Collection

Obtain long-term satellite-derived precipitation data using the CHIRPS dataset available in GEE. This data will be used to analyze drought conditions in Kenya.

### Step 2: Study Area Definition

Define the study area using Kenya's geographical boundaries.

### Step 3: Data Preprocessing

- **Extract Monthly Precipitation**: Extract monthly CHIRPS precipitation data for the study area to calculate Standardized Precipitation Index (SPI) at different time scales (e.g., SPI1, SPI3, SPI6, SPI12).
- **Clip to Region**: Clip the precipitation data to Kenya's boundaries to ensure that the analysis focuses solely on Kenya. This step has already been implemented using GEE.

### Step 4: Calculate Standardized Precipitation Index (SPI)

- **SPI Calculation**: Calculate SPI at different time scales (1-, 3-, 6-, and 12-month) for drought evaluation. Convert the CHIRPS precipitation data into SPI values by fitting a gamma distribution to each pixel's monthly time series.

### Step 5: Drought Characterization

- **Identify Drought Events**: Use run theory to identify drought events based on SPI values.
- **Drought Duration**: Identify the number of months with SPI values below thresholds like -1.0 for moderate drought.
- **Drought Severity**: Calculate the severity by summing all SPI values during the drought.
- **Drought Intensity**: Calculate drought intensity as severity divided by duration.

### Step 6: Trend Analysis

- **Mann-Kendall Test**: Implement the Mann-Kendall trend test to determine trends in SPI values at annual, seasonal, and monthly time scales.
- **Sen’s Slope Estimator**: Apply Sen’s slope estimator to understand the magnitude of the detected trends.

### Step 7: Clustering Analysis of Drought Metrics

This step aims to perform a comprehensive clustering analysis on drought metrics across administrative units in Kenya, including data preprocessing, dimensionality reduction, and clustering.

#### Workflow Steps

- **Data Preprocessing**:
  - Encode categorical features, and normalize numerical metrics using `MinMaxScaler`.
- **Feature Engineering**:
  - Construct a feature vector (`feature_vector`) for clustering, including encoded categorical features and  normalized numerical metrics.
- **Dimensionality Reduction Using PCA**:
  - Determine the optimal number of components for dimensionality reduction by analyzing cumulative explained variance and applying PCA accordingly.
- **Clustering Analysis**:
  - **K-Means Clustering**:
    - Determine the optimal number of clusters using the **Elbow Method** and apply K-Means.
    - Visualize the clusters geospatially.
  - **Hierarchical Clustering**:
    - Create a linkage matrix using **Ward's method** and determine the optimal clusters using a dendrogram.
    - Assign cluster labels and visualize geospatially.
  - **DBSCAN Clustering**:
    - Perform a grid search to determine the best parameters (`eps`, `min_samples`) and use the **Silhouette Score** for optimization.
    - Assign cluster labels and visualize geospatially.


### Step 8: EM-DAT Analysis

Compare historical drought years from the [Emergency Events Database (EM-DAT)](https://www.emdat.be/) to SPI-derived results.

### DISCLAIMER

This is a set of scripts  shared for educational purposes only.  Anyone who uses this code or its
functionality or structure, assumes full liability and credits the author.

#### Map Disclaimer

The designations employed and the presentation of the material on this map do not imply the expression
of any opinion whatsoever on the part of the author concerning the legal status of any country, territory, city or area or of its authorities, or concerning the delimitation of its
frontiers or boundaries.


# Google Earth Engine Setup

### Install Geemap

In [ ]:
!pip install -U "geemap[workshop]"

### Import libraries

In [ ]:
!pip install pymannkendall

In [3]:
import ee
import geemap
import numpy as np
import pandas as pd
from scipy.stats import gamma, norm, kstest, probplot
import time
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
from ipywidgets import interact, Dropdown, fixed, widgets, Checkbox, RadioButtons
import pymannkendall as mk
import json
import geopandas as gpd
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.metrics import silhouette_score

### Authenticate and initialize Earth Engine

You will need to create a [Google Cloud Project](https://console.cloud.google.com/projectcreate) and enable the [Earth Engine API](https://console.cloud.google.com/apis/api/earthengine.googleapis.com) for the project. You can find detailed instructions [here](https://book.geemap.org/chapters/01_introduction.html#earth-engine-authentication).

In [4]:
ee.Authenticate()

Update the project below.

In [5]:
ee.Initialize(project="bayesian-clustering-for-zoning")

## Initialize Map

In [ ]:
# Creating a map
m = geemap.Map(center=[-1.94, 29.87], zoom=8)
m

Map(center=[-1.94, 29.87], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

# Define Analysis Boundaries

We use the following [crop calendar](https://fews.net/sites/default/files/styles/large_width_880/public/2023-03/seasonal-calendar-kenya.png?itok=0Wob_hCK) to select the regions and time periods to focus on.

We define boundaries for the Western and Rift Valley, and Eastern and Northern Kenya. We use the [FAO GAUL: Global Administrative Unit Layers](https://data.apps.fao.org/catalog/dataset/global-administrative-unit-layers-gaul) for the analysis.

The Global Administrative Unit Layers (GAUL) compiles and disseminates the best available information on administrative units for all the countries in the world, providing a contribution to the standardization of the spatial dataset representing administrative units. The GAUL always maintains global layers with a unified coding system at country, first (e.g. departments), and second administrative levels (e.g. districts). Where data is available, it provides layers on a country by country basis down to third, fourth, and lowers levels.

In [6]:
# Load the FAO GAUL dataset for Kenya at administrative level 2
admin_level = 'level2'
country_name = 'Rwanda'
roi = ee.FeatureCollection(f"FAO/GAUL/2015/{admin_level}")
roi = roi.filter(ee.Filter.eq('ADM0_NAME', country_name))

# Print the available counties (ADM1_NAME) for verification
counties_list = roi.aggregate_array('ADM2_NAME').getInfo()
print(counties_list)

['Bugesera', 'Gatsibo', 'Kayonza', 'Kirehe', 'Ngoma', 'Nyagatare', 'Rwamagana', 'Gasabo', 'Kicukiro', 'Nyarugenge', 'Burera', 'Gakenke', 'Gicumbi', 'Musanze', 'Rulindo', 'Gisagara', 'Huye', 'Kamonyi', 'Muhanga', 'Nyamagabe', 'Nyanza', 'Nyaruguru', 'Ruhango', 'Karongi', 'Ngororero', 'Nyabihu', 'Nyamasheke', 'Rubavu', 'Rusizi', 'Rutsiro']


In [7]:
# Define the country and administrative level
country_name = 'Rwanda'
admin_level = 'level2'
roi = ee.FeatureCollection(f"FAO/GAUL/2015/{admin_level}").filter(ee.Filter.eq('ADM0_NAME', country_name))

# Export to GeoJSON
geojson = roi.getInfo()
with open('rwanda_admin2_boundaries.geojson', 'w') as f:
    json.dump(geojson, f)

In [8]:
# Define the new admin 2 regions for "Western and Rift Valley" and "Eastern and Northern Kenya"
east_admin2 = [
    'Bugesera', 'Gatsibo', 'Kayonza', 'Kirehe', 'Ngoma', 'Nyagatare', 'Rwamagana'
]

city_admin2 = [
    'Gasabo', 'Kicukiro', 'Nyarugenge'
]

north_admin2 = [
    'Burera', 'Gakenke', 'Gicumbi', 'Musanze', 'Rulindo'
]

south_admin2 = [
    'Gisagara', 'Huye', 'Kamonyi', 'Muhanga', 'Nyamagabe', 'Nyanza', 'Nyaruguru', 'Ruhango'
]
west_admin2 = [
    'Karongi', 'Ngororero', 'Nyabihu', 'Nyamasheke', 'Rubavu', 'Rusizi', 'Rutsiro'
]

# Filter the regions based on Admin 2 names

east_df = roi.filter(ee.Filter.inList('ADM2_NAME', east_admin2))
city_df = roi.filter(ee.Filter.inList('ADM2_NAME', city_admin2))
north_df = roi.filter(ee.Filter.inList('ADM2_NAME', north_admin2))
south_df = roi.filter(ee.Filter.inList('ADM2_NAME', south_admin2))
west_df = roi.filter(ee.Filter.inList('ADM2_NAME', west_admin2))

# Print the filtered collections for verification
print("East/Iburasirazuba (Admin 2 level):", east_df.aggregate_array('ADM2_NAME').getInfo())
print("Kigali City/Umujyi wa Kigali (Admin 2 level):", city_df.aggregate_array('ADM2_NAME').getInfo())
print("North/Amajyaruguru (Admin 2 level):", north_df.aggregate_array('ADM2_NAME').getInfo())
print("South/Amajyepfo (Admin 2 level):", south_df.aggregate_array('ADM2_NAME').getInfo())
print("West/Iburengerazuba (Admin 2 level):", west_df.aggregate_array('ADM2_NAME').getInfo())

# Adding Western & Rift Valley and Eastern & Northern Kenya as layers to the map
m.addLayer(east_df, {"color": "blue"}, "East/Iburasirazuba")
m.addLayer(city_df, {"color": "red"}, "Kigali City/Umujyi wa Kigali")
m.addLayer(north_df, {"color": "green"}, "North/Amajyaruguru")
m.addLayer(south_df, {"color": "yellow"}, "South/Amajyepfo")
m.addLayer(west_df, {"color": "orange"}, "West/Iburengerazuba")

# Visualize the boundaries
m

East/Iburasirazuba (Admin 2 level): ['Bugesera', 'Gatsibo', 'Kayonza', 'Kirehe', 'Ngoma', 'Nyagatare', 'Rwamagana']
Kigali City/Umujyi wa Kigali (Admin 2 level): ['Gasabo', 'Kicukiro', 'Nyarugenge']
North/Amajyaruguru (Admin 2 level): ['Burera', 'Gakenke', 'Gicumbi', 'Musanze', 'Rulindo']
South/Amajyepfo (Admin 2 level): ['Gisagara', 'Huye', 'Kamonyi', 'Muhanga', 'Nyamagabe', 'Nyanza', 'Nyaruguru', 'Ruhango']
West/Iburengerazuba (Admin 2 level): ['Karongi', 'Ngororero', 'Nyabihu', 'Nyamasheke', 'Rubavu', 'Rusizi', 'Rutsiro']


NameError: name 'm' is not defined

# Load CHIRPS Pentad Dataset

# Define Time Period

The periods of interest are the planting, harvest, and rain seasons for different regions of Kenya. We will use these timelines to define the start and end dates for creating the relevant dekadal (10-day period) analysis from the CHIRPS pentad data.

1. Western and Rift Valley  

- Long Rains: March to June
- Planting: April to June
- Long Rains Harvest: July to August & October to November

2. Eastern and Northern Kenya:

- Short Rains Harvest: February to March
- Planting: March to May
- Long Rains: March to June
- Lean Season: September to November
- Short Rains: October to December

Based on this calendar, we'll use the start and end months for rainfall using the CHIRPS Pentad dataset ('UCSB-CHG/CHIRPS/PENTAD').

In [ ]:
def monthly_sum_from_pentads(clipped_pentads, region_name, region, year, start_month, end_month):
    """
    Calculate monthly precipitation sums from CHIRPS pentads (5-day intervals) for a specified region (at Admin 2 level) and time period.

    Parameters:
    clipped_pentads (ee.ImageCollection): The CHIRPS pentad data clipped to the region of interest.
    region_name (str): Name of the broader region.
    region (ee.FeatureCollection): FeatureCollection of the region containing individual Admin 2 geometries.
    year (int): The year for which monthly totals are to be created.
    start_month (int): The starting month of the period (1 to 12).
    end_month (int): The ending month of the period (1 to 12).

    Returns:
    list: A list of dictionaries containing monthly precipitation data for each Admin 2 region within the specified range.
    """
    monthly_precip_data = []

    # Loop through each Admin 2 feature in the region
    admin2_features = region.toList(region.size())

    for i in range(region.size().getInfo()):
        admin2_feature = ee.Feature(admin2_features.get(i))
        admin2_name = admin2_feature.get('ADM2_NAME').getInfo()

        # Loop through the specified months to calculate monthly precipitation totals
        for month in range(start_month, end_month + 1):
            start_month_date = ee.Date.fromYMD(year, month, 1)
            end_month_date = start_month_date.advance(1, 'month')

            # Filter pentad data for the current month
            pentad_filtered = clipped_pentads.filterDate(start_month_date, end_month_date)

            # Sum all pentads to form the monthly total
            monthly_sum = pentad_filtered.sum() \
                                         .reduceRegion(
                                             reducer=ee.Reducer.mean(),
                                             geometry=admin2_feature.geometry(),
                                             scale=5000,
                                             maxPixels=1e8
                                         )

            # Extract the value and add additional information
            precipitation_value = monthly_sum.get('precipitation').getInfo() if monthly_sum.get('precipitation') else None

            monthly_precip_data.append({
                'year': year,
                'month': month,
                'date': start_month_date.format().getInfo(),
                'region': region_name,
                'admin2_name': admin2_name,
                'precipitation': precipitation_value
            })

    return monthly_precip_data

In [ ]:
# Load the CHIRPS Pentad dataset
chirps_pentad = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD')

# Clip the CHIRPS pentad dataset for each region
chirps_pentad_east = chirps_pentad.filterBounds(east_df)
chirps_pentad_city = chirps_pentad.filterBounds(city_df)
chirps_pentad_north = chirps_pentad.filterBounds(north_df)
chirps_pentad_south = chirps_pentad.filterBounds(south_df)
chirps_pentad_west = chirps_pentad.filterBounds(west_df)


# Define Inputs

long_rains_start = 2  # February
long_rains_end = 5  # May
short_rains_start = 9  # September
short_rains_end = 11  # November
start_year = 2000
end_year = 2023

# Initialize empty lists to store results for long rains and short rains
east_long_rains_results = []
east_short_rains_results = []
city_long_rains_results = []
city_short_rains_results = []
north_long_rains_results = []
north_short_rains_results = []
south_long_rains_results = []
south_short_rains_results = []
west_long_rains_results = []
west_short_rains_results = []

In [ ]:
#East

# Loop through each year from start_year to end_year
for year in range(start_year, end_year + 1):
    # Start timing for the current year
    start_time = time.time()
    print(f"Year {year} started...")

    # Long Rains (February to May)
    east_long_rains = monthly_sum_from_pentads(
        clipped_pentads=chirps_pentad_east,
        region_name="east",
        region=east_df,
        year=year,
        start_month=long_rains_start,
        end_month=long_rains_end
    )
    east_long_rains_results.extend(east_long_rains)

    # Short Rains (September to November)
    east_short_rains = monthly_sum_from_pentads(
        clipped_pentads=chirps_pentad_east,
        region_name="east",
        region=east_df,
        year=year,
        start_month=short_rains_start,
        end_month=short_rains_end
    )
    east_short_rains_results.extend(east_short_rains)

    # End timing for the current year
    end_time = time.time()
    elapsed_time = end_time - start_time
    minutes, seconds = divmod(elapsed_time, 60)

    print(f"Year {year} completed. Time taken: {int(minutes)} minutes and {int(seconds)} seconds.\n")

# Convert the results into pandas DataFrames
east_long_rains_df = pd.DataFrame(east_long_rains_results)
east_long_rains_df['Season'] = 'Long Rain'
east_short_rains_df = pd.DataFrame(east_short_rains_results)
east_short_rains_df['Season'] = 'Short Rain'

# Display the first few rows of the DataFrames
print("\nEast Long Rains:")
print(east_long_rains_df.head())

print("\nEast Short Rains:")
print(east_short_rains_df.head())

Year 2000 started...
Year 2000 completed. Time taken: 1 minutes and 56 seconds.

Year 2001 started...
Year 2001 completed. Time taken: 2 minutes and 8 seconds.

Year 2002 started...
Year 2002 completed. Time taken: 1 minutes and 56 seconds.

Year 2003 started...
Year 2003 completed. Time taken: 1 minutes and 54 seconds.

Year 2004 started...
Year 2004 completed. Time taken: 4 minutes and 28 seconds.

Year 2005 started...
Year 2005 completed. Time taken: 3 minutes and 30 seconds.

Year 2006 started...
Year 2006 completed. Time taken: 2 minutes and 54 seconds.

Year 2007 started...
Year 2007 completed. Time taken: 3 minutes and 8 seconds.

Year 2008 started...
Year 2008 completed. Time taken: 2 minutes and 33 seconds.

Year 2009 started...
Year 2009 completed. Time taken: 3 minutes and 4 seconds.

Year 2010 started...
Year 2010 completed. Time taken: 2 minutes and 41 seconds.

Year 2011 started...
Year 2011 completed. Time taken: 2 minutes and 44 seconds.

Year 2012 started...
Year 2012 

In [ ]:
#city

# Loop through each year from start_year to end_year
for year in range(start_year, end_year + 1):
    # Start timing for the current year
    start_time = time.time()
    print(f"Year {year} started...")

    # Long Rains (February to May)
    city_long_rains = monthly_sum_from_pentads(
        clipped_pentads=chirps_pentad_city,
        region_name="city",
        region=city_df,
        year=year,
        start_month=long_rains_start,
        end_month=long_rains_end
    )
    city_long_rains_results.extend(city_long_rains)

    # Short Rains (September to November)
    city_short_rains = monthly_sum_from_pentads(
        clipped_pentads=chirps_pentad_city,
        region_name="city",
        region=city_df,
        year=year,
        start_month=short_rains_start,
        end_month=short_rains_end
    )
    city_short_rains_results.extend(city_short_rains)

    # End timing for the current year
    end_time = time.time()
    elapsed_time = end_time - start_time
    minutes, seconds = divmod(elapsed_time, 60)

    print(f"Year {year} completed. Time taken: {int(minutes)} minutes and {int(seconds)} seconds.\n")

# Convert the results into pandas DataFrames
city_long_rains_df = pd.DataFrame(city_long_rains_results)
city_long_rains_df['Season'] = 'Long Rain'
city_short_rains_df = pd.DataFrame(city_short_rains_results)
city_short_rains_df['Season'] = 'Short Rain'

# Display the first few rows of the DataFrames
print("\ncity Long Rains:")
print(city_long_rains_df.head())

print("\ncity Short Rains:")
print(city_short_rains_df.head())

Year 2000 started...
Year 2000 completed. Time taken: 0 minutes and 36 seconds.

Year 2001 started...
Year 2001 completed. Time taken: 0 minutes and 33 seconds.

Year 2002 started...
Year 2002 completed. Time taken: 0 minutes and 42 seconds.

Year 2003 started...
Year 2003 completed. Time taken: 0 minutes and 50 seconds.

Year 2004 started...
Year 2004 completed. Time taken: 0 minutes and 39 seconds.

Year 2005 started...
Year 2005 completed. Time taken: 0 minutes and 32 seconds.

Year 2006 started...
Year 2006 completed. Time taken: 0 minutes and 42 seconds.

Year 2007 started...
Year 2007 completed. Time taken: 0 minutes and 39 seconds.

Year 2008 started...
Year 2008 completed. Time taken: 0 minutes and 37 seconds.

Year 2009 started...
Year 2009 completed. Time taken: 0 minutes and 31 seconds.

Year 2010 started...
Year 2010 completed. Time taken: 0 minutes and 42 seconds.

Year 2011 started...
Year 2011 completed. Time taken: 0 minutes and 35 seconds.

Year 2012 started...
Year 20

In [ ]:
#west

# Loop through each year from start_year to end_year
for year in range(start_year, end_year + 1):
    # Start timing for the current year
    start_time = time.time()
    print(f"Year {year} started...")

    # Long Rains (February to May)
    west_long_rains = monthly_sum_from_pentads(
        clipped_pentads=chirps_pentad_west,
        region_name="west",
        region=west_df,
        year=year,
        start_month=long_rains_start,
        end_month=long_rains_end
    )
    west_long_rains_results.extend(west_long_rains)

    # Short Rains (September to November)
    west_short_rains = monthly_sum_from_pentads(
        clipped_pentads=chirps_pentad_west,
        region_name="west",
        region=west_df,
        year=year,
        start_month=short_rains_start,
        end_month=short_rains_end
    )
    west_short_rains_results.extend(west_short_rains)

    # End timing for the current year
    end_time = time.time()
    elapsed_time = end_time - start_time
    minutes, seconds = divmod(elapsed_time, 60)

    print(f"Year {year} completed. Time taken: {int(minutes)} minutes and {int(seconds)} seconds.\n")

# Convert the results into pandas DataFrames
west_long_rains_df = pd.DataFrame(west_long_rains_results)
west_long_rains_df['Season'] = 'Long Rain'
west_short_rains_df = pd.DataFrame(west_short_rains_results)
west_short_rains_df['Season'] = 'Short Rain'

# Display the first few rows of the DataFrames
print("\nwest Long Rains:")
print(west_long_rains_df.head())

print("\nwest Short Rains:")
print(west_short_rains_df.head())

Year 2000 started...
Year 2000 completed. Time taken: 1 minutes and 49 seconds.

Year 2001 started...
Year 2001 completed. Time taken: 2 minutes and 1 seconds.

Year 2002 started...


Year 2002 completed. Time taken: 1 minutes and 59 seconds.

Year 2003 started...
Year 2003 completed. Time taken: 2 minutes and 15 seconds.

Year 2004 started...
Year 2004 completed. Time taken: 2 minutes and 2 seconds.

Year 2005 started...
Year 2005 completed. Time taken: 1 minutes and 58 seconds.

Year 2006 started...
Year 2006 completed. Time taken: 1 minutes and 57 seconds.

Year 2007 started...
Year 2007 completed. Time taken: 1 minutes and 52 seconds.

Year 2008 started...
Year 2008 completed. Time taken: 1 minutes and 53 seconds.

Year 2009 started...
Year 2009 completed. Time taken: 2 minutes and 6 seconds.

Year 2010 started...
Year 2010 completed. Time taken: 1 minutes and 46 seconds.

Year 2011 started...
Year 2011 completed. Time taken: 2 minutes and 30 seconds.

Year 2012 started...
Year 2012 completed. Time taken: 2 minutes and 52 seconds.

Year 2013 started...
Year 2013 completed. Time taken: 1 minutes and 51 seconds.

Year 2014 started...
Year 2014 completed. Time take

In [ ]:
#north

# Loop through each year from start_year to end_year
for year in range(start_year, end_year + 1):
    # Start timing for the current year
    start_time = time.time()
    print(f"Year {year} started...")

    # Long Rains (February to May)
    north_long_rains = monthly_sum_from_pentads(
        clipped_pentads=chirps_pentad_north,
        region_name="north",
        region=north_df,
        year=year,
        start_month=long_rains_start,
        end_month=long_rains_end
    )
    north_long_rains_results.extend(north_long_rains)

    # Short Rains (September to November)
    north_short_rains = monthly_sum_from_pentads(
        clipped_pentads=chirps_pentad_north,
        region_name="north",
        region=north_df,
        year=year,
        start_month=short_rains_start,
        end_month=short_rains_end
    )
    north_short_rains_results.extend(north_short_rains)

    # End timing for the current year
    end_time = time.time()
    elapsed_time = end_time - start_time
    minutes, seconds = divmod(elapsed_time, 60)

    print(f"Year {year} completed. Time taken: {int(minutes)} minutes and {int(seconds)} seconds.\n")

# Convert the results into pandas DataFrames
north_long_rains_df = pd.DataFrame(north_long_rains_results)
north_long_rains_df['Season'] = 'Long Rain'
north_short_rains_df = pd.DataFrame(north_short_rains_results)
north_short_rains_df['Season'] = 'Short Rain'

# Display the first few rows of the DataFrames
print("\nnorth Long Rains:")
print(north_long_rains_df.head())

print("\nnorth Short Rains:")
print(north_short_rains_df.head())

Year 2000 started...
Year 2000 completed. Time taken: 1 minutes and 14 seconds.

Year 2001 started...
Year 2001 completed. Time taken: 0 minutes and 48 seconds.

Year 2002 started...
Year 2002 completed. Time taken: 0 minutes and 37 seconds.

Year 2003 started...
Year 2003 completed. Time taken: 0 minutes and 37 seconds.

Year 2004 started...
Year 2004 completed. Time taken: 0 minutes and 35 seconds.

Year 2005 started...
Year 2005 completed. Time taken: 0 minutes and 33 seconds.

Year 2006 started...
Year 2006 completed. Time taken: 0 minutes and 37 seconds.

Year 2007 started...
Year 2007 completed. Time taken: 0 minutes and 34 seconds.

Year 2008 started...
Year 2008 completed. Time taken: 0 minutes and 33 seconds.

Year 2009 started...
Year 2009 completed. Time taken: 0 minutes and 38 seconds.

Year 2010 started...
Year 2010 completed. Time taken: 0 minutes and 34 seconds.

Year 2011 started...
Year 2011 completed. Time taken: 0 minutes and 33 seconds.

Year 2012 started...
Year 20

In [ ]:
#south

# Loop through each year from start_year to end_year
for year in range(start_year, end_year + 1):
    # Start timing for the current year
    start_time = time.time()
    print(f"Year {year} started...")

    # Long Rains (February to May)
    south_long_rains = monthly_sum_from_pentads(
        clipped_pentads=chirps_pentad_south,
        region_name="south",
        region=south_df,
        year=year,
        start_month=long_rains_start,
        end_month=long_rains_end
    )
    south_long_rains_results.extend(south_long_rains)

    # Short Rains (September to November)
    south_short_rains = monthly_sum_from_pentads(
        clipped_pentads=chirps_pentad_south,
        region_name="south",
        region=south_df,
        year=year,
        start_month=short_rains_start,
        end_month=short_rains_end
    )
    south_short_rains_results.extend(south_short_rains)

    # End timing for the current year
    end_time = time.time()
    elapsed_time = end_time - start_time
    minutes, seconds = divmod(elapsed_time, 60)

    print(f"Year {year} completed. Time taken: {int(minutes)} minutes and {int(seconds)} seconds.\n")

# Convert the results into pandas DataFrames
south_long_rains_df = pd.DataFrame(south_long_rains_results)
south_long_rains_df['Season'] = 'Long Rain'
south_short_rains_df = pd.DataFrame(south_short_rains_results)
south_short_rains_df['Season'] = 'Short Rain'

# Display the first few rows of the DataFrames
print("\nsouth Long Rains:")
print(south_long_rains_df.head())

print("\nsouth Short Rains:")
print(south_short_rains_df.head())

In [ ]:
# Combine all three DataFrames into a single DataFrame
combined_df = pd.concat([
    east_long_rains_df,
    east_short_rains_df,
    city_long_rains_df,
    city_short_rains_df,
    west_long_rains_df,
    west_short_rains_df,
    north_long_rains_df,
    north_short_rains_df,
    south_long_rains_df,
    south_short_rains_df
])

# Reset index after concatenation
combined_df.reset_index(drop=True, inplace=True)

# Save the file
combined_df.to_csv("/content/rwanda_admin2_precip.csv")

In [13]:
combined_df

,Unnamed: 0,year,month,date,region,admin2_name,precipitation,Season
0,0,2000,2,2000-02-01T00:00:00,east,Bugesera,43.557124,Long Rain
1,1,2000,3,2000-03-01T00:00:00,east,Bugesera,116.565669,Long Rain
2,2,2000,4,2000-04-01T00:00:00,east,Bugesera,87.765666,Long Rain
3,3,2000,5,2000-05-01T00:00:00,east,Bugesera,50.155928,Long Rain
4,4,2000,2,2000-02-01T00:00:00,east,Gatsibo,45.895979,Long Rain
...,...,...,...,...,...,...,...,...
5035,5035,2023,10,2023-10-01T00:00:00,south,Nyaruguru,131.889331,Short Rain
5036,5036,2023,11,2023-11-01T00:00:00,south,Nyaruguru,163.599941,Short Rain
5037,5037,2023,9,2023-09-01T00:00:00,south,Ruhango,78.141782,Short Rain
5038,5038,2023,10,2023-10-01T00:00:00,south,Ruhango,83.234868,Short Rain


# Calculate Standardized Precipitation Index (SPI)

## SPI Definitions

* SPI1 (1-month scale)

Used to capture short-term monthly precipitation fluctuations and early warning of meteorological drought.

* SPI3 (3-month scale)

Useful for seasonal drought analysis, which helps in understanding the drought conditions over a quarterly period, closely related to agricultural impacts.

* SPI6 (6-month scale)

Provides insights into medium-term drought conditions, capturing both the end of one season and the start of another, which can influence soil moisture and crop yield over a longer period.

* SPI12 (12-month scale)

Used to assess long-term drought conditions, representing annual fluctuations and providing insights into the overall hydrological drought scenario, which can affect groundwater recharge and surface water availability.


## Selecting SPI Time Scales
Based on the above definitions we select the following time Scales for Long and Short Rains:

- Long Rains: March to June (4-month period) - We will use 1-month and 3-month SPI for long rains to understand short-term and medium-term droughts.
- Short Rains: October to December (3-month period) - We will use 1-month and 3-month SPI to capture variability within the shorter rainy season.

## Determining Gamma Distribution

What is a Gamma Distribution?

A gamma distribution is a type of statistical model used to describe data that are always positive and usually skewed, meaning most values are small but there can be a few large values. It's often used for things like rainfall, where:

- Many days have little or no rain, and
- A few days have a lot of rain.

This type of pattern creates a graph that has a high peak near the small values and a long tail extending to larger values. That’s what the gamma distribution looks like.

Why Do We Use Gamma Distribution for Rainfall?

- Always Positive: Rainfall cannot be negative; it either doesn’t rain or it rains some amount. The gamma distribution is great for representing only positive values.
- Right-Skewed Data: Most of the time, we have small amounts of rainfall, and occasionally, we get heavy rainfall. The gamma distribution is good for representing this kind of uneven distribution.
- Flexibility: The shape of the gamma distribution can change to fit different types of rainfall patterns, making it a very flexible tool for modeling different weather conditions.

The Gamma Distribution is the Standardized Precipitation Index (SPI) as a way to figure out if a place is having normal, dry, or wet weather compared to its history.

1. Modeling Historical Rainfall:

We use the gamma distribution to fit the historical rainfall data. This helps us understand what the usual rainfall is like for each month.

2. Comparing to Current Rainfall:

Once we have the gamma model of typical rainfall, we compare current rainfall to see how much it differs from the usual.
This comparison tells us if it is drier or wetter than normal, and by how much.

3. Getting SPI:

The final step is to convert this difference into a number called SPI, which tells us:
- If the SPI is negative, it means it’s drier than usual (possible drought).
- If the SPI is positive, it means it’s wetter than usual (possible flooding).

In [15]:
# Checking if all precipitation values are positive in each dataset
print("East Long Rains: All values positive? ", (east_long_rains_df['precipitation'] > 0).all())
print("East Short Rains: All values positive? ", (east_short_rains_df['precipitation'] > 0).all())

print("City Long Rains: All values positive? ", (city_long_rains_df['precipitation'] > 0).all())
print("City Short Rains: All values positive? ", (city_short_rains_df['precipitation'] > 0).all())

print("West Long Rains: All values positive? ", (west_long_rains_df['precipitation'] > 0).all())
print("West Short Rains: All values positive? ", (west_short_rains_df['precipitation'] > 0).all())

print("North Long Rains: All values positive? ", (north_long_rains_df['precipitation'] > 0).all())
print("North Short Rains: All values positive? ", (north_short_rains_df['precipitation'] > 0).all())

print("South Long Rains: All values positive? ", (south_long_rains_df['precipitation'] > 0).all())
print("South Short Rains: All values positive? ", (south_short_rains_df['precipitation'] > 0).all())

East Long Rains: All values positive?  True
East Short Rains: All values positive?  True
City Long Rains: All values positive?  True
City Short Rains: All values positive?  True
West Long Rains: All values positive?  True
West Short Rains: All values positive?  True
North Long Rains: All values positive?  True
North Short Rains: All values positive?  True
South Long Rains: All values positive?  True
South Short Rains: All values positive?  True


### Histograms

In [ ]:
# This loaded dataframe is created using GEE
combined_df = pd.read_csv("data/rwanda_admin2_precip.csv.csv")

# Adding an additional column to each DataFrame to identify the type of rains
# East long rains
east_long_rains_df = combined_df[
    (combined_df['region'] == 'east') &
    (combined_df['Season'] == 'Long Rain')
]

# East short rains
east_short_rains_df = combined_df[
    (combined_df['region'] == 'east') &
    (combined_df['Season'] == 'Short Rain')
]

# City long rains
city_long_rains_df = combined_df[
    (combined_df['region'] == 'city') &
    (combined_df['Season'] == 'Long Rain')
]

# City short rains
city_short_rains_df = combined_df[
    (combined_df['region'] == 'city') &
    (combined_df['Season'] == 'Short Rain')
]

# North long rains
north_long_rains_df = combined_df[
    (combined_df['region'] == 'north') &
    (combined_df['Season'] == 'Long Rain')
]

# North short rains
north_short_rains_df = combined_df[
    (combined_df['region'] == 'north') &
    (combined_df['Season'] == 'Short Rain')
]

# South long rains
south_long_rains_df = combined_df[
    (combined_df['region'] == 'south') &
    (combined_df['Season'] == 'Long Rain')
]

# South short rains
south_short_rains_df = combined_df[
    (combined_df['region'] == 'south') &
    (combined_df['Season'] == 'Short Rain')
]
# West long rains
west_long_rains_df = combined_df[
    (combined_df['region'] == 'west') &
    (combined_df['Season'] == 'Long Rain')
]

# West short rains
west_short_rains_df = combined_df[
    (combined_df['region'] == 'west') &
    (combined_df['Season'] == 'Short Rain')
]

# Adding an additional column to each DataFrame to identify the type of rains

east_long_rains_df['season'] = 'Long Rains'
east_short_rains_df['season'] = 'Short Rains'

city_long_rains_df['season'] = 'Long Rains'
city_short_rains_df['season'] = 'Short Rains'

north_long_rains_df['season'] = 'Long Rains'
north_short_rains_df['season'] = 'Short Rains'

south_long_rains_df['season'] = 'Long Rains'
south_short_rains_df['season'] = 'Short Rains'

west_long_rains_df['season'] = 'Long Rains'
west_short_rains_df['season'] = 'Short Rains'

# Combine all three DataFrames into a single DataFrame
combined_df = pd.concat([
    east_long_rains_df,
    east_short_rains_df,
    city_long_rains_df,
    city_short_rains_df,
    west_long_rains_df,
    west_short_rains_df,
    north_long_rains_df,
    north_short_rains_df,
    south_long_rains_df,
    south_short_rains_df
])

# Reset index after concatenation
combined_df.reset_index(drop=True, inplace=True)

In [22]:
# Define an interactive function for plotting histograms
def interactive_histogram(region, admin2_name, season):
    """
    Plots a histogram of the precipitation data for a specific region, Admin 2 area, and season.

    Parameters:
    region (str): The selected region.
    admin2_name (str): The selected Admin 2 area within the region.
    season (str): The selected season (e.g., Long Rains or Short Rains).
    """
    # Filter the DataFrame based on the selected region, admin2_name, and season
    filtered_df = combined_df[
        (combined_df['region'] == region) &
        (combined_df['admin2_name'] == admin2_name) &
        (combined_df['season'] == season)
    ]

    if filtered_df.empty:
        print("No data available for the selected region, admin2_name, and season.")
        return

    # Plot histogram
    plt.figure(figsize=(10, 6))
    plt.hist(filtered_df['precipitation'], bins=20, color='skyblue', edgecolor='black')
    plt.title(f'Precipitation Histogram for {admin2_name} ({region}, {season})')
    plt.xlabel('Precipitation')
    plt.ylabel('Frequency')
    plt.grid(axis='y', alpha=0.75)
    plt.show()

# Define the interactive dropdown widgets
region_dropdown = Dropdown(
    options=combined_df['region'].unique(),
    description='Select Region:',
    style={'description_width': 'initial'}
)

admin2_name_dropdown = Dropdown(
    options=[],  # Initially empty, will be updated based on region selection
    description='Select Admin2:',
    style={'description_width': 'initial'}
)

season_dropdown = Dropdown(
    options=combined_df['season'].unique(),
    description='Select Season:',
    style={'description_width': 'initial'}
)

# Function to update admin2_name options based on the selected region
def update_admin2_dropdown(change):
    selected_region = change['new']
    if selected_region:
        admin2_options = combined_df[combined_df['region'] == selected_region]['admin2_name'].unique()
        admin2_name_dropdown.options = admin2_options
    else:
        admin2_name_dropdown.options = []

# Attach an observer to the region dropdown to update the admin2_name dropdown
region_dropdown.observe(update_admin2_dropdown, names='value')

# Set an initial value to trigger the observer and populate admin2_name dropdown
update_admin2_dropdown({'new': region_dropdown.value})

# Use `interact` to create an interactive histogram based on the selected region, admin2_name, and season
interact(interactive_histogram, region=region_dropdown, admin2_name=admin2_name_dropdown, season=season_dropdown)

interactive(children=(Dropdown(description='Select Region:', options=('east', 'city', 'west', 'north', 'south'…

<function __main__.interactive_histogram(region, admin2_name, season)>

### Kolmogorov-Smirnov Test

Understanding the Kolmogorov-Smirnov Test:
The Kolmogorov-Smirnov test compares the empirical distribution of your data to a theoretical distribution (in this case, the gamma distribution).
The null hypothesis (H₀) of the KS test is that the data follows the specified distribution (in this case, gamma).
The alternative hypothesis (H₁) is that the data does not follow the specified distribution.

In the context of the Kolmogorov-Smirnov (KS) test, a p-value greater than 0.05 is typically desired when checking the fit of a distribution.

Interpreting the p-value:

- p-value > 0.05:
If the p-value is greater than 0.05, it means there is insufficient evidence to reject the null hypothesis.
In other words, the data fits the gamma distribution well. Therefore, we accept the null hypothesis and conclude that the gamma distribution is likely a good fit.

- p-value < 0.05:
If the p-value is less than 0.05, it means that there is significant evidence to reject the null hypothesis.
This suggests that the gamma distribution may not be a good fit for the data.

In [23]:
# Function to perform Kolmogorov-Smirnov test for each admin2_name and determine if it passes or fails
def perform_ks_test_for_all(region, season, pass_or_fail="pass"):
    """
    Perform Kolmogorov-Smirnov test for each admin2_name within the selected region and season.

    Parameters:
    region (str): Name of the region.
    season (str): Name of the season (e.g., 'Long Rains', 'Short Rains').
    pass_or_fail (str): Specify whether to print areas that pass or fail the test ('pass' or 'fail').

    Returns:
    None: Prints the names of admin2 areas that pass or fail the KS test.
    """
    # Filter the dataset based on the provided region and season
    filtered_df = combined_df[(combined_df['region'] == region) & (combined_df['season'] == season)]

    # Initialize lists to hold the names of areas that pass or fail the test
    passing_admin2_names = []
    failing_admin2_names = []

    # Loop through each admin2_name in the filtered DataFrame
    for admin2_name in filtered_df['admin2_name'].unique():
        # Get the precipitation data for the current admin2_name
        data = filtered_df[filtered_df['admin2_name'] == admin2_name]['precipitation'].dropna()

        # Check if there is enough data to perform the test
        if len(data) < 2:
            continue

        # Fit a gamma distribution to the data
        shape, loc, scale = gamma.fit(data, floc=0)  # floc=0 to ensure non-negative values

        # Perform Kolmogorov-Smirnov test
        _, p_value = kstest(data, gamma(shape, loc, scale).cdf)

        # Determine if the admin2_name passes or fails based on the p-value
        if p_value > 0.05:
            passing_admin2_names.append(admin2_name)
        else:
            failing_admin2_names.append(admin2_name)

    # Print the results based on the pass_or_fail argument
    if pass_or_fail == "pass":
        if passing_admin2_names:
            print(f"Admin2 areas in {region} during {season} that pass the KS test:")
            for name in passing_admin2_names:
                print(name)
        else:
            print(f"No admin2 areas in {region} during {season} passed the KS test.")
    elif pass_or_fail == "fail":
        if failing_admin2_names:
            print(f"Admin2 areas in {region} during {season} that fail the KS test:")
            for name in failing_admin2_names:
                print(name)
        else:
            print(f"No admin2 areas in {region} during {season} failed the KS test.")
    else:
        print("Invalid argument for 'pass_or_fail'. Please use 'pass' or 'fail'.")

# Interactive widgets
region_dropdown = Dropdown(
    options=combined_df['region'].unique(),
    description='Select Region:',
    style={'description_width': 'initial'}
)

season_dropdown = Dropdown(
    options=combined_df['season'].unique(),
    description='Select Season:',
    style={'description_width': 'initial'}
)

pass_or_fail_dropdown = Dropdown(
    options=['pass', 'fail'],
    description='Pass or Fail:',
    style={'description_width': 'initial'}
)

# Function to update the Admin2 dropdown options based on the selected region
def update_admin2_dropdown(region):
    admin2_options = combined_df[combined_df['region'] == region]['admin2_name'].unique()
    admin2_name_dropdown.options = admin2_options

# Interactive function to display the results of the KS test for all admin2 areas
interact(
    perform_ks_test_for_all,
    region=region_dropdown,
    season=season_dropdown,
    pass_or_fail=pass_or_fail_dropdown
)


interactive(children=(Dropdown(description='Select Region:', options=('east', 'city', 'west', 'north', 'south'…

<function __main__.perform_ks_test_for_all(region, season, pass_or_fail='pass')>

### QQ Plots

A QQ plot, or Quantile-Quantile plot, is a graphical tool used to compare the distribution of a dataset with a theoretical distribution, such as the normal or gamma distribution. It's a provides a visual assessment of how well the dataset fits a chosen probability distribution. The QQ plot compares the quantiles of the dataset with the quantiles of the specified theoretical distribution.

If the data follows the theoretical distribution closely, the points on the plot will align approximately along a straight line. Deviations from this line indicate that the data does not conform well to the expected distribution.

In [24]:
# Define an interactive function for generating QQ plots
def interactive_qq_plot(region, admin2_name, season):
    """
    Generates a QQ plot for the selected dataset using a gamma distribution.

    Parameters:
    region (str): The selected region.
    admin2_name (str): The selected Admin 2 area within the region.
    season (str): The selected season (e.g., Long Rains or Short Rains).
    """
    # Filter the DataFrame based on the selected region, admin2_name, and season
    filtered_df = combined_df[
        (combined_df['region'] == region) &
        (combined_df['admin2_name'] == admin2_name) &
        (combined_df['season'] == season)
    ]

    if filtered_df.empty:
        print("No data available for the selected region, admin2_name, and season.")
        return

    data = filtered_df['precipitation'].dropna()

    # Fit the gamma distribution to the data
    shape, loc, scale = gamma.fit(data, floc=0)

    # Generate QQ plot
    plt.figure(figsize=(8, 6))
    probplot(data, dist="gamma", sparams=(shape, loc, scale), plot=plt)
    plt.title(f"QQ Plot to Check Gamma Fit: {admin2_name} ({region}, {season})")
    plt.xlabel("Theoretical Quantiles")
    plt.ylabel("Ordered Values")
    plt.grid(True)
    plt.show()

# Define the interactive dropdown widgets
region_dropdown = Dropdown(
    options=combined_df['region'].unique(),
    description='Select Region:',
    style={'description_width': 'initial'}
)

admin2_name_dropdown = Dropdown(
    options=[],  # Initially empty, will be updated based on the region selected
    description='Select Admin2:',
    style={'description_width': 'initial'}
)

season_dropdown = Dropdown(
    options=combined_df['season'].unique(),
    description='Select Season:',
    style={'description_width': 'initial'}
)

# Function to update admin2_name options based on the selected region
def update_admin2_dropdown(change):
    selected_region = change['new']
    if selected_region:
        admin2_options = combined_df[combined_df['region'] == selected_region]['admin2_name'].unique()
        admin2_name_dropdown.options = admin2_options
    else:
        admin2_name_dropdown.options = []

# Attach an observer to the region dropdown to update the admin2_name dropdown
region_dropdown.observe(update_admin2_dropdown, names='value')

# Set an initial value to trigger the observer and populate admin2_name dropdown
update_admin2_dropdown({'new': region_dropdown.value})

# Use `interact` to create an interactive QQ plot based on the selected region, admin2_name, and season
interact(interactive_qq_plot, region=region_dropdown, admin2_name=admin2_name_dropdown, season=season_dropdown)

interactive(children=(Dropdown(description='Select Region:', options=('east', 'city', 'west', 'north', 'south'…

<function __main__.interactive_qq_plot(region, admin2_name, season)>

## Calculate SPI

In [25]:
# Combine all datasets
all_rains_df = combined_df

# Convert the 'date' column to datetime
all_rains_df['date'] = pd.to_datetime(all_rains_df['date'])

# Set the 'date' as the index for easier time series operations
all_rains_df.set_index('date', inplace=True)

# Sort the data by index
all_rains_df.sort_index(inplace=True)

In [26]:
all_rains_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5040 entries, 2000-02-01 to 2023-11-01
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     5040 non-null   int64  
 1   year           5040 non-null   int64  
 2   month          5040 non-null   int64  
 3   region         5040 non-null   object 
 4   admin2_name    5040 non-null   object 
 5   precipitation  5040 non-null   float64
 6   Season         5040 non-null   object 
 7   season         5040 non-null   object 
dtypes: float64(1), int64(3), object(4)
memory usage: 354.4+ KB


In [27]:
def calculate_spi(precip_series, scale=1):
    """
    Calculate Standardized Precipitation Index (SPI) for a given precipitation series.

    Parameters:
    precip_series (pd.Series): Monthly precipitation data.
    scale (int): Time scale over which to calculate SPI (e.g., 1-month, 3-month).

    Returns:
    pd.Series: SPI values for the given precipitation series.
    """
    # Calculate rolling sum over the specified time scale
    precip_rolling = precip_series.rolling(window=scale).sum()

    # Drop NaN values resulting from the rolling operation
    precip_rolling = precip_rolling.dropna()

    # Fit a gamma distribution to the rolling sum data
    shape, loc, scale_param = gamma.fit(precip_rolling, floc=0)

    # Calculate cumulative distribution function (CDF) for gamma distribution
    gamma_cdf = gamma.cdf(precip_rolling, shape, loc=loc, scale=scale_param)

    # Convert gamma CDF to standard normal distribution to calculate SPI
    spi_values = norm.ppf(gamma_cdf)

    # Return the SPI values as a pandas Series with the same index as the rolling sum
    return pd.Series(spi_values, index=precip_rolling.index)

# Prepare the data by grouping it by region, admin2_name, and season, and calculating SPI separately
spi_results_list = []

for (region, admin2_name, season), group in all_rains_df.groupby(['region', 'admin2_name', 'season']):
    precip_series = group['precipitation']

    # Calculate 1-month SPI
    spi_1_month = calculate_spi(precip_series, scale=1)

    # Create a DataFrame from the SPI result for 1-month scale
    spi_1_month_df = pd.DataFrame({
        'year': spi_1_month.index.year,
        'month': spi_1_month.index.month,
        'region': region,
        'admin2_name': admin2_name,
        'season': season,
        'precipitation': precip_series.loc[spi_1_month.index],
        'spi': spi_1_month,
        'spi_scale': '1'  # Indicates the 1-month SPI
    })

    # Append to the results list
    spi_results_list.append(spi_1_month_df)

    # Calculate 3-month SPI
    spi_3_month = calculate_spi(precip_series, scale=3)

    # Create a DataFrame from the SPI result for 3-month scale
    spi_3_month_df = pd.DataFrame({
        'year': spi_3_month.index.year,
        'month': spi_3_month.index.month,
        'region': region,
        'admin2_name': admin2_name,
        'season': season,
        'precipitation': precip_series.loc[spi_3_month.index],
        'spi': spi_3_month,
        'spi_scale': '3'  # Indicates the 3-month SPI
    })

    # Append to the results list
    spi_results_list.append(spi_3_month_df)

# Concatenate all SPI results into a single DataFrame
spi_results_df = pd.concat(spi_results_list)

# Reset the index for the final DataFrame
spi_results_df.reset_index(drop=True, inplace=True)

# Display the first few rows of the final DataFrame
print(spi_results_df.head())

   year  month region admin2_name      season  precipitation       spi  \
0  2000      2   city      Gasabo  Long Rains      48.453012 -1.667118   
1  2000      3   city      Gasabo  Long Rains     105.644945 -0.196961   
2  2000      4   city      Gasabo  Long Rains      96.106213 -0.394784   
3  2000      5   city      Gasabo  Long Rains      72.906209 -0.939720   
4  2001      2   city      Gasabo  Long Rains      62.269462 -1.230800   

  spi_scale  
0         1  
1         1  
2         1  
3         1  
4         1  


In [28]:
# Function to plot SPI values
def plot_spi(region, admin2_name, season, spi_scale):
    """
    Plots the Standardized Precipitation Index (SPI) for the selected region, Admin 2 area, season, and scale.

    Parameters:
    region (str): Selected region (e.g., Western Rift).
    admin2_name (str): Selected Admin 2 area within the region.
    season (str): Selected season (e.g., Long Rains, Short Rains).
    spi_scale (str): Selected SPI scale (e.g., 1-month, 3-month).
    """
    # Filter the DataFrame based on the selected region, admin2_name, season, and spi_scale
    filtered_df = spi_results_df[
        (spi_results_df['region'] == region) &
        (spi_results_df['admin2_name'] == admin2_name) &
        (spi_results_df['season'] == season) &
        (spi_results_df['spi_scale'] == spi_scale)
    ]

    if filtered_df.empty:
        print(f"No data available for {admin2_name} in {region} for {season} with {spi_scale}-month SPI.")
        return

    # Create a datetime index from the year and month columns
    filtered_df['date'] = pd.to_datetime(filtered_df[['year', 'month']].assign(day=1))

    # Extract the SPI series with the datetime index
    spi_series = filtered_df.set_index('date')['spi'].dropna()

    # Plot the SPI values
    plt.figure(figsize=(10, 6))
    plt.plot(spi_series.index, spi_series, color='blue', linestyle='-', marker='o', label='SPI')
    plt.axhline(y=0, color='black', linestyle='--')  # Reference line at SPI=0
    plt.axhline(y=-1, color='red', linestyle='--', label='Mild Drought (SPI=-1)')
    plt.axhline(y=-1.5, color='orange', linestyle='--', label='Moderate Drought (SPI=-1.5)')
    plt.axhline(y=-2, color='darkred', linestyle='--', label='Severe Drought (SPI=-2)')

    plt.xlabel('Date')
    plt.ylabel('SPI Value')
    plt.title(f'Standardized Precipitation Index (SPI) for {admin2_name} in {region} ({season}, {spi_scale}-Month Scale)')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Create interactive dropdown widgets
region_dropdown = Dropdown(
    options=spi_results_df['region'].unique(),
    description='Select Region:',
    style={'description_width': 'initial'}
)

admin2_name_dropdown = Dropdown(
    description='Select Admin2:',
    style={'description_width': 'initial'}
)

season_dropdown = Dropdown(
    options=spi_results_df['season'].unique(),
    description='Select Season:',
    style={'description_width': 'initial'}
)

spi_scale_dropdown = Dropdown(
    options=['1', '3'],  # Options for 1-month and 3-month SPI
    description='Select SPI Scale (months):',
    style={'description_width': 'initial'}
)

# Function to update the Admin2 dropdown options based on the selected region
def update_admin2_dropdown(change):
    selected_region = change['new']
    if selected_region:
        admin2_options = spi_results_df[spi_results_df['region'] == selected_region]['admin2_name'].unique().tolist()
        admin2_name_dropdown.options = admin2_options
    else:
        admin2_name_dropdown.options = []

# Attach the update function to the 'region' dropdown
region_dropdown.observe(update_admin2_dropdown, names='value')

# Set initial value for admin2_name_dropdown when the widget is first displayed
update_admin2_dropdown({'new': region_dropdown.value})

# Use `interact` to create an interactive plot for SPI values based on the selected parameters
interact(
    plot_spi,
    region=region_dropdown,
    admin2_name=admin2_name_dropdown,
    season=season_dropdown,
    spi_scale=spi_scale_dropdown
)

interactive(children=(Dropdown(description='Select Region:', options=('city', 'east', 'north', 'south', 'west'…

<function __main__.plot_spi(region, admin2_name, season, spi_scale)>

## Identify Drought Events

To characterize droughts, we need to analyze the SPI values calculated previously to identify distinct drought events. This involves analyzing when SPI values fall below a threshold, like -1.0, to determine the duration, severity, and intensity of each drought event.

Definitions:

- Drought Event: A period during which SPI is below a defined threshold (e.g., SPI < -1.0 for moderate drought).
- Drought Duration: The number of months during which the SPI value stays below the threshold.
- Drought Severity: The sum of all SPI values during the drought period.
- Drought Intensity: Severity divided by duration, indicating how intense the drought is on average.

In [29]:
def characterize_drought_events(df, spi_threshold=-1.0):
    """
    Characterize drought events based on SPI values.

    Parameters:
    df (pd.DataFrame): Input DataFrame containing columns:
                       'year', 'month', 'region', 'admin2_name', 'season', 'precipitation', 'spi', 'spi_scale'.
    spi_threshold (float): Threshold for defining a drought event (default is -1.0 for moderate drought).

    Returns:
    pd.DataFrame: Output DataFrame containing columns:
                  'region', 'admin2_name', 'season', 'spi_scale', 'Drought Event', 'Drought Duration',
                  'Drought Severity', 'Drought Intensity', 'Drought Event_start', 'Drought Event_end'.
    """
    drought_events = []

    # Group the data by region, admin2_name, season, and spi_scale
    grouped = df.groupby(['region', 'admin2_name', 'season', 'spi_scale'])

    for (region, admin2_name, season, spi_scale), group in grouped:
        group = group.sort_values(by=['year', 'month'])
        in_drought = False
        start_date = None
        severity = 0
        drought_event_count = 0

        # Loop through each row in the group to identify drought events
        for _, row in group.iterrows():
            if row['spi'] < spi_threshold:
                if not in_drought:
                    # Start of a new drought event
                    in_drought = True
                    start_date = f"{row['year']}-{row['month']:02d}"
                    severity = 0
                    drought_event_count += 1

                # Accumulate severity
                severity += row['spi']
            else:
                if in_drought:
                    # End of the drought event
                    in_drought = False
                    end_date = f"{row['year']}-{row['month']:02d}"
                    duration = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days // 30
                    intensity = severity / duration if duration > 0 else 0

                    # Append drought event details
                    drought_events.append({
                        'region': region,
                        'admin2_name': admin2_name,
                        'season': season,
                        'spi_scale': spi_scale,
                        'Drought Event': drought_event_count,
                        'Drought Duration': duration,
                        'Drought Severity': severity,
                        'Drought Intensity': intensity,
                        'Drought Event_start': start_date,
                        'Drought Event_end': end_date
                    })

        # Handle case where a drought event is ongoing at the end of the data
        if in_drought:
            end_date = f"{row['year']}-{row['month']:02d}"
            duration = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days // 30
            intensity = severity / duration if duration > 0 else 0

            drought_events.append({
                'region': region,
                'admin2_name': admin2_name,
                'season': season,
                'spi_scale': spi_scale,
                'Drought Event': drought_event_count,
                'Drought Duration': duration,
                'Drought Severity': severity,
                'Drought Intensity': intensity,
                'Drought Event_start': start_date,
                'Drought Event_end': end_date
            })

    # Convert the list of drought events to a DataFrame
    drought_events_df = pd.DataFrame(drought_events)

    return drought_events_df

drought_events_df = characterize_drought_events(spi_results_df)
drought_events_df

,region,admin2_name,season,spi_scale,Drought Event,Drought Duration,Drought Severity,Drought Intensity,Drought Event_start,Drought Event_end
0,city,Gasabo,Long Rains,1,1,0,-1.667118,0.000000,2000-02,2000-03
1,city,Gasabo,Long Rains,1,2,0,-1.230800,0.000000,2001-02,2001-03
2,city,Gasabo,Long Rains,1,3,0,-1.373133,0.000000,2003-02,2003-03
3,city,Gasabo,Long Rains,1,4,9,-1.457783,-0.161976,2004-05,2005-02
4,city,Gasabo,Long Rains,1,5,0,-1.079836,0.000000,2008-02,2008-03
...,...,...,...,...,...,...,...,...,...,...
1149,west,Rutsiro,Short Rains,3,3,12,-6.092539,-0.507712,2005-11,2006-11
1150,west,Rutsiro,Short Rains,3,4,11,-3.214600,-0.292236,2008-11,2009-10
1151,west,Rutsiro,Short Rains,3,5,12,-3.937792,-0.328149,2013-11,2014-11
1152,west,Rutsiro,Short Rains,3,6,1,-1.006830,-1.006830,2022-09,2022-10


Note - The issue where drought durations exceed expected limits (e.g., durations exceeding 3 months for a "3-month SPI scale") arises from the way the drought characterization function is designed. Specifically, the function might be identifying prolonged drought events that span beyond a single season or multi-month drought events that are not being properly split when SPI values rise above the threshold.

Due to continuous negative SPI Values, the function currently identifies a drought event as continuing until the SPI rises above the threshold. If the SPI remains below the threshold for an extended period (even beyond a season), the function will count that entire period as a single drought event.

### Distribution Analysis

In [30]:
# Interactive plotting function
def plot_interactive_histogram(region, admin2_name, season, variable, df=drought_events_df):
    """
    Plots an interactive histogram or density plot for a selected drought characteristic.

    Parameters:
    - region (str): Selected region.
    - admin2_name (str): Selected admin2_name (administrative unit).
    - season (str): Selected season.
    - variable (str): The drought characteristic to plot (e.g., 'Drought Duration', 'Drought Severity', 'Drought Intensity').
    - df (pd.DataFrame): The DataFrame containing drought characteristics data.
    """
    # Filter data based on user selections
    filtered_df = df[
        (df['region'] == region) &
        (df['admin2_name'] == admin2_name) &
        (df['season'] == season)
    ]

    # Check if filtered data is not empty
    if filtered_df.empty:
        print("No data available for the selected filters.")
        return

    # Plot the histogram
    plt.figure(figsize=(10, 6))
    sns.histplot(filtered_df[variable], bins=20, kde=True, color='skyblue', edgecolor='black')
    plt.title(f"Distribution of {variable} for {region} ({admin2_name}) - {season}")
    plt.xlabel(variable)
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Dropdown options
region_options = drought_events_df['region'].unique().tolist()
admin2_name_options = drought_events_df['admin2_name'].unique().tolist()
season_options = drought_events_df['season'].unique().tolist()
variable_options = ['Drought Duration', 'Drought Severity', 'Drought Intensity']

# Create dropdown widgets
region_dropdown = Dropdown(options=region_options, description='Region:', style={'description_width': 'initial'})
admin2_name_dropdown = Dropdown(options=admin2_name_options, description='Admin2 Name:', style={'description_width': 'initial'})
season_dropdown = Dropdown(options=season_options, description='Season:', style={'description_width': 'initial'})
variable_dropdown = Dropdown(options=variable_options, description='Variable:', style={'description_width': 'initial'})

# Function to update the Admin2 dropdown based on the selected region
def update_admin2_options(*args):
    selected_region = region_dropdown.value
    filtered_admin2_options = drought_events_df[drought_events_df['region'] == selected_region]['admin2_name'].unique().tolist()
    admin2_name_dropdown.options = filtered_admin2_options

# Attach the update function to the 'region' dropdown
region_dropdown.observe(update_admin2_options, names='value')

# Create interactive dropdown widgets for selecting region, admin2_name, season, and variable
interact(
    plot_interactive_histogram,
    region=region_dropdown,
    admin2_name=admin2_name_dropdown,
    season=season_dropdown,
    variable=variable_dropdown,
    df=fixed(drought_events_df)  # Pass the DataFrame as a fixed value
)

interactive(children=(Dropdown(description='Region:', options=('city', 'east', 'north', 'south', 'west'), styl…

<function __main__.plot_interactive_histogram(region, admin2_name, season, variable, df=     region admin2_name       season spi_scale  Drought Event  \
0      city      Gasabo   Long Rains         1              1   
1      city      Gasabo   Long Rains         1              2   
2      city      Gasabo   Long Rains         1              3   
3      city      Gasabo   Long Rains         1              4   
4      city      Gasabo   Long Rains         1              5   
...     ...         ...          ...       ...            ...   
1149   west     Rutsiro  Short Rains         3              3   
1150   west     Rutsiro  Short Rains         3              4   
1151   west     Rutsiro  Short Rains         3              5   
1152   west     Rutsiro  Short Rains         3              6   
1153   west     Rutsiro  Short Rains         3              7   

      Drought Duration  Drought Severity  Drought Intensity  \
0                    0         -1.667118           0.000000   
1                    0         -1.230800           0.000000   
2                    0         -1.373133           0.000000   
3                    9         -1.457783          -0.161976   
4                    0         -1.079836           0.000000   
...                ...               ...                ...   
1149                12         -6.092539          -0.507712   
1150                11         -3.214600          -0.292236   
1151                12         -3.937792          -0.328149   
1152                 1         -1.006830          -1.006830   
1153                 0         -1.252744           0.000000   

     Drought Event_start Drought Event_end  
0                2000-02           2000-03  
1                2001-02           2001-03  
2                2003-02           2003-03  
3                2004-05           2005-02  
4                2008-02           2008-03  
...                  ...               ...  
1149             2005-11           2006-11  
1150             2008-11           2009-10  
1151             2013-11           2014-11  
1152             2022-09           2022-10  
1153             2023-11           2023-11  

[1154 rows x 10 columns])>

### Drought Event Frequency

In [31]:
# Define an interactive function to plot drought event frequency
def plot_drought_event_frequency(df, dimension, sort_by_height):
    """
    Plots a bar chart for drought event frequency based on the selected dimension,
    with 'region' as the color of the bars.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing drought characteristics data.
    - dimension (str): The dimension to group by ('admin2_name', 'season', 'spi_scale').
    - sort_by_height (bool): Whether to sort the bars based on frequency.
    """
    # Check if the selected dimension is in the DataFrame
    if dimension not in df.columns:
        print(f"Dimension '{dimension}' is not available in the DataFrame.")
        return

    # Group by the selected dimension and region to calculate the frequency of drought events
    drought_freq = df.groupby([dimension, 'region'])['Drought Event'].nunique().reset_index()
    drought_freq = drought_freq.rename(columns={'Drought Event': 'Drought Event Frequency'})

    # Sort values if specified
    if sort_by_height:
        drought_freq = drought_freq.sort_values(by='Drought Event Frequency', ascending=False)

    # Plot the bar chart
    plt.figure(figsize=(12, 6))
    sns.barplot(
        x=dimension,
        y='Drought Event Frequency',
        hue='region',
        data=drought_freq,
        palette=['#1f77b4', '#ff7f0e']  # Blue and orange colors
    )

    plt.title(f'Frequency of Drought Events by {dimension.capitalize()} (Colored by Region)')
    plt.xlabel(dimension.capitalize())
    plt.ylabel('Drought Event Frequency')
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Region', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.grid(axis='y', alpha=0.75)
    plt.show()

# Define the dropdown options for different dimensions (excluding 'region')
dimension_options = ['admin2_name', 'season', 'spi_scale']

# Create interactive dropdown widgets for selecting the dimension and sort by height
interact(
    plot_drought_event_frequency,
    df=fixed(drought_events_df),
    dimension=Dropdown(
        options=dimension_options,
        description='Select Dimension:',
        style={'description_width': 'initial'}
    ),
    sort_by_height=Checkbox(
        value=False,
        description='Sort by Height',
        style={'description_width': 'initial'}
    )
)


interactive(children=(Dropdown(description='Select Dimension:', options=('admin2_name', 'season', 'spi_scale')…

<function __main__.plot_drought_event_frequency(df, dimension, sort_by_height)>

### Regional Comparison

In [32]:
# Define an interactive function to compare drought characteristics across regions and administrative levels
def plot_drought_characteristics(df, characteristic):
    """
    Plots a boxplot to compare drought characteristics across regions and administrative units.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing drought characteristics data.
    - characteristic (str): The drought characteristic to plot ('Drought Intensity', 'Drought Severity', 'Drought Duration').
    """
    # Check if the selected characteristic is in the DataFrame
    if characteristic not in df.columns:
        print(f"Characteristic '{characteristic}' is not available in the DataFrame.")
        return

    # Set up the plot
    plt.figure(figsize=(14, 7))

    # Plot the boxplot for the selected characteristic
    sns.boxplot(
        x='region',
        y=characteristic,
        hue='admin2_name',
        data=df,
        palette='Blues'
    )

    plt.title(f'Boxplot of {characteristic} by Region and Admin2 Level')
    plt.xlabel('Region')
    plt.ylabel(characteristic)
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Admin2 Name', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.grid(axis='y', alpha=0.75)
    plt.show()

# Define the dropdown options for different characteristics
characteristic_options = ['Drought Intensity', 'Drought Severity', 'Drought Duration']

# Create interactive dropdown widgets for selecting characteristic
interact(
    plot_drought_characteristics,
    df=fixed(drought_events_df),
    characteristic=Dropdown(
        options=characteristic_options,
        description='Select Characteristic:',
        style={'description_width': 'initial'}
    )
)


interactive(children=(Dropdown(description='Select Characteristic:', options=('Drought Intensity', 'Drought Se…

<function __main__.plot_drought_characteristics(df, characteristic)>

### Correlation Analysis

In [33]:
# Define an interactive function to perform correlation analysis
def plot_correlation_analysis(df, x_variable, y_variable):
    """
    Plots a scatter plot to analyze the correlation between two selected drought characteristics.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing drought characteristics data.
    - x_variable (str): The x-axis drought characteristic to plot.
    - y_variable (str): The y-axis drought characteristic to plot.
    """
    # Check if the selected variables are in the DataFrame
    if x_variable not in df.columns or y_variable not in df.columns:
        print(f"One or both selected characteristics are not available in the DataFrame.")
        return

    # Set up the plot
    plt.figure(figsize=(10, 6))

    # Plot the scatter plot for the selected characteristics
    sns.scatterplot(
        x=x_variable,
        y=y_variable,
        data=df,
        hue='region',
        palette=['blue', 'orange'],
        alpha=0.7
    )

    plt.title(f'Scatter Plot of {y_variable} vs {x_variable}')
    plt.xlabel(x_variable)
    plt.ylabel(y_variable)
    plt.grid(True)
    plt.legend(title='Region', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

# Define the dropdown options for different characteristics
correlation_options = ['Drought Duration', 'Drought Severity', 'Drought Intensity']

# Create interactive dropdown widgets for selecting x and y variables
interact(
    plot_correlation_analysis,
    df=fixed(drought_events_df),
    x_variable=Dropdown(
        options=correlation_options,
        description='Select X Variable:',
        style={'description_width': 'initial'}
    ),
    y_variable=Dropdown(
        options=correlation_options,
        description='Select Y Variable:',
        style={'description_width': 'initial'}
    )
)

interactive(children=(Dropdown(description='Select X Variable:', options=('Drought Duration', 'Drought Severit…

<function __main__.plot_correlation_analysis(df, x_variable, y_variable)>

### SPI Scale (1-month vs. 3-month) Analysis

In [34]:
# Define an interactive function for seasonal drought analysis
def plot_drought_analysis(df, characteristic, analysis_category):
    """
    Plots a grouped bar chart to compare drought characteristics based on the selected analysis category.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing drought characteristics data.
    - characteristic (str): The drought characteristic to plot (e.g., 'Drought Duration', 'Drought Severity', 'Drought Intensity').
    - analysis_category (str): The category to analyze (e.g., 'season' or 'region').
    """
    # Group the DataFrame by the selected analysis category and 'spi_scale', and calculate the mean for the selected characteristic
    grouped_df = df.groupby([analysis_category, 'spi_scale'])[characteristic].mean().reset_index()

    # Set up the plot
    plt.figure(figsize=(12, 6))

    # Plot a grouped bar chart for the selected characteristic
    sns.barplot(
        x=analysis_category,
        y=characteristic,
        hue='spi_scale',
        data=grouped_df,
        palette=['blue', 'orange'],
        edgecolor='black'
    )

    plt.title(f'Comparison of {characteristic} and {analysis_category.title()} by SPI Scale')
    plt.xlabel(analysis_category.title())
    plt.ylabel(f'Mean {characteristic}')
    plt.grid(axis='y', alpha=0.75)
    plt.tight_layout()
    plt.legend(title='SPI Scale')
    plt.show()

# Define the dropdown options for drought characteristics and analysis category
characteristic_options = ['Drought Duration', 'Drought Severity', 'Drought Intensity']
analysis_category_options = ['season', 'region']

# Use `interact` to create an interactive grouped bar chart based on the selected characteristic and category
interact(
    plot_drought_analysis,
    df=fixed(drought_events_df),
    characteristic=Dropdown(
        options=characteristic_options,
        description='Select Characteristic:',
        style={'description_width': 'initial'}
    ),
    analysis_category=Dropdown(
        options=analysis_category_options,
        description='Analyze By:',
        style={'description_width': 'initial'}
    )
)


interactive(children=(Dropdown(description='Select Characteristic:', options=('Drought Duration', 'Drought Sev…

<function __main__.plot_drought_analysis(df, characteristic, analysis_category)>

### Temporal Heatmaps

In [35]:
# Define an interactive function for temporal heatmap analysis
def plot_temporal_heatmap(df, region, characteristic):
    """
    Plots a heatmap to show drought events over time for a specific region.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing drought characteristics data.
    - region (str): The selected region to visualize.
    - characteristic (str): The drought characteristic to plot (e.g., 'Drought Duration', 'Drought Severity').
    """
    # Filter the DataFrame based on the selected region
    filtered_df = df[df['region'] == region]

    # Convert 'Drought Event_start' to datetime if not already
    if not pd.api.types.is_datetime64_any_dtype(filtered_df['Drought Event_start']):
        filtered_df['Drought Event_start'] = pd.to_datetime(filtered_df['Drought Event_start'])

    # Create a pivot table with years on the x-axis and months on the y-axis for the selected characteristic
    filtered_df['year'] = filtered_df['Drought Event_start'].dt.year
    filtered_df['month'] = filtered_df['Drought Event_start'].dt.month

    pivot_table = filtered_df.pivot_table(
        index='month',
        columns='year',
        values=characteristic,
        aggfunc='mean',
        fill_value=0
    )

    # Plot the heatmap
    plt.figure(figsize=(12, 8))
    sns.heatmap(pivot_table, cmap='Blues', annot=True, fmt='.1f', linewidths=.5, cbar_kws={'label': characteristic})

    plt.title(f'Temporal Heatmap of Mean {characteristic} for {region}')
    plt.xlabel('Year')
    plt.ylabel('Month')
    plt.tight_layout()
    plt.show()

# Define the dropdown options for region and drought characteristics
region_options = drought_events_df['region'].unique().tolist()
characteristic_options = ['Drought Duration', 'Drought Severity', 'Drought Intensity']

# Use `interact` to create an interactive heatmap based on the selected region and characteristic
interact(
    plot_temporal_heatmap,
    df=fixed(drought_events_df),
    region=Dropdown(
        options=region_options,
        description='Select Region:',
        style={'description_width': 'initial'}
    ),
    characteristic=Dropdown(
        options=characteristic_options,
        description='Select Characteristic:',
        style={'description_width': 'initial'}
    )
)

interactive(children=(Dropdown(description='Select Region:', options=('city', 'east', 'north', 'south', 'west'…

<function __main__.plot_temporal_heatmap(df, region, characteristic)>

### Drought Event Analysis

In [36]:
# Define an interactive function for drought event analysis
def plot_drought_event_analysis(region, admin2_name, characteristic):
    """
    Plots drought event analysis for the selected region, admin2_name, and characteristic.

    Parameters:
    - region (str): The selected region.
    - admin2_name (str): The selected Admin 2 area within the region.
    - characteristic (str): The characteristic to plot (e.g., 'Drought Duration', 'Drought Event Frequency', 'Drought Severity').
    """
    # Filter the DataFrame based on the selected region and admin2_name
    filtered_df = drought_events_df[
        (drought_events_df['region'] == region) &
        (drought_events_df['admin2_name'] == admin2_name)
    ]

    # Ensure columns 'Drought Event_start' and 'Drought Event_end' are in datetime format
    filtered_df['Drought Event_start'] = pd.to_datetime(filtered_df['Drought Event_start'])
    filtered_df['Drought Event_end'] = pd.to_datetime(filtered_df['Drought Event_end'])

    # Extract year from the start date
    filtered_df['year'] = filtered_df['Drought Event_start'].dt.year

    # Group the data by year and calculate event frequency or average characteristic
    if characteristic == 'Drought Event Frequency':
        drought_analysis = filtered_df.groupby('year').size().reset_index(name='Event Frequency')
    elif characteristic == 'Average Drought Duration':
        drought_analysis = filtered_df.groupby('year')['Drought Duration'].mean().reset_index(name='Average Duration')
    elif characteristic == 'Average Drought Severity':
        drought_analysis = filtered_df.groupby('year')['Drought Severity'].mean().reset_index(name='Average Severity')

    # Plot the drought event analysis
    plt.figure(figsize=(12, 6))
    if characteristic == 'Drought Event Frequency':
        plt.bar(drought_analysis['year'], drought_analysis['Event Frequency'], color='skyblue')
        plt.ylabel('Number of Drought Events')
    elif characteristic == 'Average Drought Duration':
        plt.plot(drought_analysis['year'], drought_analysis['Average Duration'], color='orange', marker='o', linestyle='-')
        plt.ylabel('Average Drought Duration (months)')
    elif characteristic == 'Average Drought Severity':
        plt.plot(drought_analysis['year'], drought_analysis['Average Severity'], color='green', marker='o', linestyle='-')
        plt.ylabel('Average Drought Severity')

    plt.xlabel('Year')
    plt.title(f'{characteristic} for {admin2_name} in {region}')
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

# Define the dropdown options
region_options = drought_events_df['region'].unique().tolist()

# Create region dropdown
region_dropdown = Dropdown(
    options=region_options,
    description='Select Region:',
    style={'description_width': 'initial'}
)

# Admin2 dropdown, updated when the region changes
admin2_name_dropdown = Dropdown(
    description='Select Admin2:',
    style={'description_width': 'initial'}
)

# Function to update admin2_name options based on the selected region
def update_admin2_dropdown(region):
    admin2_options = drought_events_df[drought_events_df['region'] == region]['admin2_name'].unique()
    admin2_name_dropdown.options = admin2_options

# Update the Admin2 dropdown options when the region is selected
def on_region_change(change):
    if change['name'] == 'value' and change['new']:
        update_admin2_dropdown(change['new'])

# Observe changes in the region dropdown
region_dropdown.observe(on_region_change, names='value')

# Use `interact` to create an interactive plot based on the selected parameters
interact(
    plot_drought_event_analysis,
    region=region_dropdown,
    admin2_name=admin2_name_dropdown,
    characteristic=Dropdown(
        options=['Drought Event Frequency', 'Average Drought Duration', 'Average Drought Severity'],
        description='Select Characteristic:',
        style={'description_width': 'initial'}
    )
)

interactive(children=(Dropdown(description='Select Region:', options=('city', 'east', 'north', 'south', 'west'…

<function __main__.plot_drought_event_analysis(region, admin2_name, characteristic)>

### Geospatial Analysis

In [45]:
# Load the geographic boundaries for admin2 regions (replace with your file path)
admin_boundaries = gpd.read_file('rwanda_admin2_boundaries.geojson')


# Merge the drought events DataFrame with the geospatial data
drought_gdf = admin_boundaries.merge(drought_events_df, left_on='ADM2_NAME', right_on='admin2_name')

DataSourceError: Failed to read GeoJSON data

In [40]:
import os
print(os.getcwd())

/content


In [42]:
assert os.path.exists('rwanda_admin2_boundaries.geojson'), "File not found!"

In [ ]:
drought_gdf.columns

In [43]:
with open('rwanda_admin2_boundaries.geojson', 'r') as f:
    data = f.read()
    print(data[:500])  # Print the first 500 characters to inspect the structure


{"type": "FeatureCollection", "columns": {"ADM0_CODE": "Integer", "ADM0_NAME": "String", "ADM1_CODE": "Integer", "ADM1_NAME": "String", "ADM2_CODE": "Integer", "ADM2_NAME": "String", "DISP_AREA": "String", "EXP2_YEAR": "Integer", "STATUS": "String", "STR2_YEAR": "Integer", "Shape_Area": "Float", "Shape_Leng": "Float", "system:index": "String"}, "version": 1701682634012831, "id": "FAO/GAUL/2015/level2", "properties": {"system:asset_size": 316969462}, "features": [{"type": "Feature", "geometry": {


In [ ]:
# Function to plot a geospatial drought map using GeoPandas and Matplotlib
def plot_geospatial_drought_map_static(characteristic):
    """
    Plots a static geospatial map showing the selected drought characteristic.

    Parameters:
    - characteristic (str): The drought characteristic to plot (e.g., 'Drought Duration', 'Drought Severity', 'Drought Intensity').
    """
    # Check if the characteristic exists in the GeoDataFrame
    if characteristic not in drought_gdf.columns:
        print(f"Error: The column '{characteristic}' does not exist in the GeoDataFrame.")
        return

    # Set up the figure and axis
    fig, ax = plt.subplots(figsize=(12, 10))

    # Plot the GeoDataFrame
    drought_gdf.plot(
        column=characteristic,  # Column to be visualized
        cmap='YlOrRd',          # Color map
        linewidth=0.8,          # Line width for boundaries
        ax=ax,                  # Axis to plot on
        edgecolor='black',      # Edge color
        legend=True             # Include a legend
    )

    # Customize the title and remove axis
    plt.title(f'{characteristic} by Administrative Unit', fontsize=15)
    plt.axis('off')

    # Display the map
    plt.show()

# Create the GeoDataFrame by merging the drought events DataFrame with the geospatial data
drought_gdf = admin_boundaries.merge(drought_events_df, left_on='ADM2_NAME', right_on='admin2_name')

# Get the available characteristics for plotting
available_characteristics = ['Drought Duration', 'Drought Severity', 'Drought Intensity']

# Use interact to create an interactive dropdown for the selected characteristic
interact(
    plot_geospatial_drought_map_static,
    characteristic=Dropdown(
        options=available_characteristics,
        description='Characteristic:',
        style={'description_width': 'initial'}
    )
)

## Trend Analysis

The Mann-Kendall test is a non-parametric test used to identify trends in time series data. It is particularly useful for climate data, like SPI values, to determine if there's a significant increasing or decreasing trend in drought conditions over time. We will also use Sen’s slope estimator to determine the magnitude of the trends identified.

In [ ]:
def perform_trend_analysis(df, region, admin2_name, season, time_scale):
    """
    Perform Mann-Kendall test and Sen's Slope Estimation for the given SPI series,
    filtered by region, admin2_name, and season.

    Parameters:
    df (pd.DataFrame): DataFrame containing SPI data with columns for region, admin2_name, season, and SPI values.
    region (str): The selected region.
    admin2_name (str): The selected administrative unit.
    season (str): The selected season.
    time_scale (str): Time scale (e.g., 'annual', 'seasonal', 'monthly').

    Returns:
    dict: Dictionary containing region, admin2_name, season, time_scale, trend, p-value, and Sen's slope.
    """
    # Filter the DataFrame based on region, admin2_name, and season
    filtered_df = df[
        (df['region'] == region) &
        (df['admin2_name'] == admin2_name) &
        (df['season'] == season)
    ]

    # Extract the SPI series and drop NaN values
    spi_series = filtered_df['spi'].dropna()

    if spi_series.empty:
        #print(f"No SPI data available for {admin2_name} in {region} during {season} season.")
        return None

    # Perform Mann-Kendall Trend Test
    mk_result = mk.original_test(spi_series)

    # Extract trend, p-value, and Sen's slope
    trend = mk_result.trend
    p_value = mk_result.p
    sen_slope = mk_result.slope

    return {
        'region': region,
        'admin2_name': admin2_name,
        'season': season,
        'time_scale': time_scale,
        'trend': trend,
        'p_value': p_value,
        'sen_slope': sen_slope
    }

# Use Function
results = []

# Define the options for trend analysis
regions = spi_results_df['region'].unique()
seasons = spi_results_df['season'].unique()
admin2_names = spi_results_df['admin2_name'].unique()
time_scales = ['monthly', 'seasonal']

# Loop through all combinations of region, admin2_name, and season
for region in regions:
    for admin2_name in admin2_names:
        for season in seasons:
            for time_scale in time_scales:
                result = perform_trend_analysis(spi_results_df, region, admin2_name, season, time_scale)
                if result:
                    results.append(result)

# Display the trend analysis results
print("Trend Analysis Results (Mann-Kendall Test and Sen's Slope):")

# Convert the results into a DataFrame for easier visualization and analysis
trend_analysis_df = pd.DataFrame(results)

# Drop rows where the 'trend' column is 'no trend'
filtered_trend_analysis_df = trend_analysis_df[trend_analysis_df['trend'] != 'no trend']

# Display the final filtered DataFrame
print(filtered_trend_analysis_df)

# Clustering Analysis

## Functions

In [ ]:
def plot_pca_variance_explained(feature_vector):
    """
    Applies PCA on the given feature vector and plots the cumulative explained variance
    against the number of components.

    Parameters:
    - feature_vector (pd.DataFrame or np.ndarray): The feature matrix to apply PCA on.

    Returns:
    - pca: The fitted PCA object (to access components, variance, etc., later).
    - cumulative_variance: The cumulative explained variance ratio for each component.
    """
    # Apply PCA without specifying the number of components to capture all variance
    pca = PCA()
    pca.fit(feature_vector)

    # Calculate the cumulative explained variance ratio
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)

    # Plot the cumulative explained variance ratio
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='-', color='b')
    plt.xlabel('Number of Components')
    plt.ylabel('Cumulative Explained Variance')
    plt.title('Explained Variance vs. Number of Components')
    plt.grid(True)
    plt.show()

    return pca, cumulative_variance

In [ ]:
def plot_elbow_method(reduced_features, cluster_range=range(1, 11)):
    """
    Plots the Elbow Method graph to determine the optimal number of clusters for K-Means.

    Parameters:
    - reduced_features (pd.DataFrame or np.ndarray): The feature matrix for clustering.
    - cluster_range (range, optional): The range of clusters to test. Default is range(1, 11).

    Returns:
    - wcss (list): A list of Within-Cluster Sum of Squares (WCSS) for each number of clusters.
    """
    # Calculate WCSS for different values of n_clusters
    wcss = []
    for n_clusters in cluster_range:
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        kmeans.fit(reduced_features)
        wcss.append(kmeans.inertia_)  # Inertia is the WCSS

    # Plot the Elbow Method graph
    plt.figure(figsize=(10, 6))
    plt.plot(cluster_range, wcss, marker='o', linestyle='-', color='b')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Within-Cluster Sum of Squares (WCSS)')
    plt.title('Elbow Method to Determine Optimal Number of Clusters')
    plt.xticks(cluster_range)
    plt.grid(True)
    plt.show()

    return wcss

In [ ]:
# Function to plot a geospatial map showing all K-Means clusters with a legend
def plot_all_geospatial_kmeans_clusters_with_legend(title):
    """
    Plots a static geospatial map showing all K-Means clusters with each cluster in a different color.
    Includes a legend to identify each cluster.
    """
    # Check if the cluster column exists in the GeoDataFrame
    if 'cluster' not in drought_gdf.columns:
        print(f"Error: The column 'cluster' does not exist in the GeoDataFrame.")
        return

    # Set up the figure and axis
    fig, ax = plt.subplots(figsize=(12, 10))

    # Plot the entire GeoDataFrame with a default color (to show background regions)
    drought_gdf.plot(
        color='lightgrey',  # Default color for non-clustered regions
        linewidth=0.5,      # Line width for boundaries
        ax=ax,              # Axis to plot on
        edgecolor='black'   # Edge color
    )

    # Get unique clusters
    unique_clusters = drought_gdf['cluster'].unique()

    # Generate distinct colors using Tableau Colors (a set of distinct colors provided by matplotlib)
    distinct_colors = list(mcolors.TABLEAU_COLORS.values())
    if len(unique_clusters) > len(distinct_colors):
        # If there are more clusters than available distinct colors, repeat the color set
        distinct_colors = distinct_colors * (len(unique_clusters) // len(distinct_colors) + 1)

    # Create a list to hold legend entries
    legend_patches = []

    # Plot each cluster with a different color
    for idx, cluster_label in enumerate(unique_clusters):
        color = distinct_colors[idx]  # Get a distinct color for the current cluster
        drought_gdf[drought_gdf['cluster'] == cluster_label].plot(
            color=color,         # Color for the current cluster
            linewidth=0.8,       # Line width for boundaries
            ax=ax,               # Axis to plot on
            edgecolor='black'    # Edge color
        )
        # Add an entry to the legend
        legend_patches.append(mpatches.Patch(color=color, label=f'Cluster {cluster_label}'))

    # Add the legend to the plot
    plt.legend(handles=legend_patches, title="K-Means Clusters", loc='upper right', fontsize='small', title_fontsize='medium')

    # Customize the title and remove axis
    plt.title(f'K-Means Clusters by Administrative Unit Using Precipitation - {title}', fontsize=15)
    plt.axis('off')

    # Display the map
    plt.show()

In [ ]:
def plot_hierarchical_dendrogram(reduced_features, method='ward', truncate_mode='level', p=5, cut_off=None, title='Dendrogram for Hierarchical Clustering'):
    """
    Performs hierarchical clustering using the specified method and plots the dendrogram.

    Parameters:
    - reduced_features (pd.DataFrame or np.ndarray): The feature matrix for clustering.
    - method (str): The linkage method to use (default is 'ward').
    - truncate_mode (str): Truncation mode for the dendrogram (default is 'level').
    - p (int): The number of levels to show in the dendrogram (default is 5).
    - cut_off (float, optional): A y-axis value where a horizontal line is drawn (to visualize a potential cut-off).
    - title (str): The title for the dendrogram plot.

    Returns:
    - linkage_matrix: The linkage matrix produced by hierarchical clustering.
    """
    # Perform hierarchical clustering using the specified method
    linkage_matrix = linkage(reduced_features, method=method)

    # Plot the dendrogram
    plt.figure(figsize=(12, 8))
    dendrogram(linkage_matrix, truncate_mode=truncate_mode, p=p)
    plt.xlabel('Sample Index or Cluster Size')
    plt.ylabel('Distance')
    plt.title(title)

    # Optional cut-off line for determining clusters
    if cut_off is not None:
        plt.axhline(y=cut_off, color='r', linestyle='--')

    # Show the plot
    plt.show()

    return linkage_matrix

In [ ]:
# Function to plot a geospatial map showing all Hierarchical Clusters with a legend
def plot_all_hierarchical_clusters_with_legend(title):
    """
    Plots a static geospatial map showing all Hierarchical Clusters with each cluster in a different color.
    Includes a legend to identify each cluster.
    """
    # Check if the hierarchical_cluster column exists in the GeoDataFrame
    if 'hierarchical_cluster' not in drought_gdf.columns:
        print(f"Error: The column 'hierarchical_cluster' does not exist in the GeoDataFrame.")
        return

    # Set up the figure and axis
    fig, ax = plt.subplots(figsize=(12, 10))

    # Plot the entire GeoDataFrame with a default color (to show background regions)
    drought_gdf.plot(
        color='lightgrey',     # Default color for all administrative units
        linewidth=0.5,         # Line width for boundaries
        ax=ax,                 # Axis to plot on
        edgecolor='black'      # Edge color
    )

    # Get unique clusters
    unique_clusters = drought_gdf['hierarchical_cluster'].unique()

    # Generate distinct colors using Tableau Colors (a set of distinct colors provided by matplotlib)
    distinct_colors = list(mcolors.TABLEAU_COLORS.values())
    if len(unique_clusters) > len(distinct_colors):
        # If there are more clusters than available distinct colors, repeat the color set
        distinct_colors = distinct_colors * (len(unique_clusters) // len(distinct_colors) + 1)

    # Create a list to hold legend entries
    legend_patches = []

    # Plot each cluster with a different color
    for idx, cluster_label in enumerate(unique_clusters):
        color = distinct_colors[idx]  # Get a distinct color for the current cluster
        drought_gdf[drought_gdf['hierarchical_cluster'] == cluster_label].plot(
            color=color,         # Color for the current cluster
            linewidth=0.8,       # Line width for boundaries
            ax=ax,               # Axis to plot on
            edgecolor='black'    # Edge color
        )
        # Add an entry to the legend
        legend_patches.append(mpatches.Patch(color=color, label=f'Cluster {cluster_label}'))

    # Add the legend to the plot
    plt.legend(handles=legend_patches, title="Hierarchical Clusters", loc='upper right', fontsize='small', title_fontsize='medium')

    # Customize the title and remove axis
    plt.title(f'Hierarchical Clusters by Administrative Unit - {title}', fontsize=15)
    plt.axis('off')

    # Display the map
    plt.show()

In [ ]:
def optimize_dbscan(df, reduced_features, eps_values=np.arange(0.1, 5.0, 0.5), min_samples_values=range(2, 15)):
    """
    Optimizes the DBSCAN parameters (eps and min_samples) using Silhouette Score, and applies DBSCAN with the best parameters.

    Parameters:
    - df (pd.DataFrame): The original DataFrame where the cluster labels will be added.
    - reduced_features (pd.DataFrame or np.ndarray): The feature matrix for clustering.
    - eps_values (np.ndarray or list): Range of eps values to test (default is np.arange(0.1, 5.0, 0.5)).
    - min_samples_values (range or list): Range of min_samples values to test (default is range(2, 15)).

    Returns:
    - df_with_clusters (pd.DataFrame): The DataFrame with an added 'dbscan_optimized_cluster' column containing the cluster labels.
    - best_eps (float): The best eps value.
    - best_min_samples (int): The best min_samples value.
    - best_score (float): The best silhouette score.
    """
    best_score = -1
    best_eps = None
    best_min_samples = None

    # Loop through all combinations of eps and min_samples
    for eps in eps_values:
        for min_samples in min_samples_values:
            # Apply DBSCAN with the current parameter combination
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            labels = dbscan.fit_predict(reduced_features)

            # Check if there are more than 1 cluster (excluding noise) and no noise labels (-1)
            if len(set(labels)) > 1 and -1 not in set(labels):
                try:
                    score = silhouette_score(reduced_features, labels)
                    # Update best parameters if the current score is better
                    if score > best_score:
                        best_score = score
                        best_eps = eps
                        best_min_samples = min_samples
                except Exception as e:
                    print(f"Error calculating silhouette score for eps={eps}, min_samples={min_samples}: {e}")

    # Check if we found a valid combination
    if best_eps is None or best_min_samples is None:
        print("No valid clusters were found with the given parameters.")
        return df, None, None, None
    else:
        # Print the best parameters and the corresponding silhouette score
        print(f"Best eps: {best_eps}")
        print(f"Best min_samples: {best_min_samples}")
        print(f"Best Silhouette Score: {best_score}")

        # Apply DBSCAN with the best parameters
        dbscan_optimized = DBSCAN(eps=best_eps, min_samples=best_min_samples)
        dbscan_labels_optimized = dbscan_optimized.fit_predict(reduced_features)

        # Add the optimized DBSCAN cluster labels to the original dataframe
        df_with_clusters = df.copy()
        df_with_clusters['dbscan_optimized_cluster'] = dbscan_labels_optimized

        # Display the first few rows of the dataframe with the optimized DBSCAN cluster assignments
        print(df_with_clusters[['dbscan_optimized_cluster']].head())

        return df_with_clusters, best_eps, best_min_samples, best_score

In [ ]:
def process_drought_dataset_by_index(datasets, index):
    """
    Processes the dataset from the dictionary based on the given index.
    Converts 'Drought Event_start' and 'Drought Event_end' to datetime format,
    extracts the start and end years, and returns the updated DataFrame.

    Parameters:
    - datasets (dict): A dictionary of datasets.
    - index (int): The index of the dataset to process (based on the dictionary order).

    Returns:
    - df (pd.DataFrame): The processed DataFrame with 'start_year' and 'end_year' columns added.
    """
    # Get the name of the dataset using the provided index
    dataset_name = list(datasets.keys())[index]

    # Access the selected dataset
    df = datasets[dataset_name].copy()

    # Convert 'Drought Event_start' and 'Drought Event_end' to datetime format (YYYY-MM)
    df['Drought Event_start'] = pd.to_datetime(df['Drought Event_start'], format='%Y-%m')
    df['Drought Event_end'] = pd.to_datetime(df['Drought Event_end'], format='%Y-%m')

    # Extract the years from 'Drought Event_start' and 'Drought Event_end'
    df['start_year'] = df['Drought Event_start'].dt.year
    df['end_year'] = df['Drought Event_end'].dt.year

    # Return the processed DataFrame
    return df

In [ ]:
import pandas as pd
from scipy import stats

# Function to handle multiple modes by picking the first mode
def mode_first(series):
    """Returns the first mode if there are multiple modes, ensuring a single value."""
    return series.mode().iloc[0] if not series.mode().empty else np.nan

def process_drought_data(df, duration_stat='median', severity_stat='median', intensity_stat='median'):
    """
    Groups the DataFrame by 'admin2_name' and calculates the mean, median, or mode of the relevant drought columns
    based on user input. Creates a binary matrix indicating the presence of drought events by year for each region
    and combines it with the grouped statistics.

    Parameters:
    - df (pd.DataFrame): The processed DataFrame containing drought metrics, drought event start and end years, and regions.
    - duration_stat (str): Statistic to apply to 'Drought Duration' ('mean', 'median', 'mode').
    - severity_stat (str): Statistic to apply to 'Drought Severity' ('mean', 'median', 'mode').
    - intensity_stat (str): Statistic to apply to 'Drought Intensity' ('mean', 'median', 'mode').

    Returns:
    - df_combined (pd.DataFrame): A DataFrame containing the computed drought metrics and the binary event matrix.
    """

    # Define a helper function to apply mean, median, or mode
    def apply_statistic(group, column, stat):
        if stat == 'mean':
            return group[column].mean()
        elif stat == 'median':
            return group[column].median()
        elif stat == 'mode':
            return mode_first(group[column])
        else:
            raise ValueError(f"Invalid statistic '{stat}' for {column}. Choose from 'mean', 'median', or 'mode'.")

    # Group by 'admin2_name' and calculate the statistics for each column
    grouped_stats = df.groupby('admin2_name').apply(lambda group: pd.Series({
        'Drought Duration': apply_statistic(group, 'Drought Duration', duration_stat),
        'Drought Severity': apply_statistic(group, 'Drought Severity', severity_stat),
        'Drought Intensity': apply_statistic(group, 'Drought Intensity', intensity_stat)
    }))

    # Create a list of years to create a binary presence matrix
    years = list(range(df['start_year'].min(), df['end_year'].max() + 1))

    # Initialize the binary event matrix
    binary_matrix = pd.DataFrame(0, index=df['admin2_name'].unique(), columns=years)

    # Populate the binary matrix: For each region, mark '1' in the years when it experienced a drought
    for i, row in df.iterrows():
        admin = row['admin2_name']
        for year in range(row['start_year'], row['end_year'] + 1):
            if year in binary_matrix.columns:
                binary_matrix.at[admin, year] = 1

    # Combine the binary matrix with the grouped statistics
    df_combined = grouped_stats.merge(binary_matrix, left_on='admin2_name', right_index=True)

    # Reset index to make 'admin2_name' a regular column
    df_combined = df_combined.reset_index()

    return df_combined

In [ ]:
def plot_drought_metrics_histograms_with_stats(df):
    """
    Plots histograms for Drought Duration, Drought Severity, and Drought Intensity.
    Adds lines for mean, median, and mode.
    First, general histograms for the entire dataset, and then individual histograms for each admin2_name.

    Parameters:
    - df (pd.DataFrame): The processed DataFrame with drought metrics and admin2_name.
    """
    # Define the columns to plot
    metrics = ['Drought Duration', 'Drought Severity', 'Drought Intensity']

    # Function to calculate mean, median, and mode and add lines to the plot
    def add_statistics_lines(data, color='r'):
        mean_val = np.mean(data)
        median_val = np.median(data)
        mode_val = stats.mode(data, keepdims=True)[0][0]  # Explicitly setting keepdims to True

        # Add lines for mean, median, and mode
        plt.axvline(mean_val, color='b', linestyle='--', label=f'Mean: {mean_val:.2f}')
        plt.axvline(median_val, color='g', linestyle='-', label=f'Median: {median_val:.2f}')
        plt.axvline(mode_val, color='r', linestyle='-', label=f'Mode: {mode_val:.2f}')
        plt.legend()

    # General histograms for the entire dataset
    plt.figure(figsize=(12, 8))
    for i, metric in enumerate(metrics, 1):
        plt.subplot(3, 1, i)
        sns.histplot(df[metric].replace([np.inf, -np.inf], np.nan).dropna(), bins=20, kde=True)
        plt.title(f'Histogram for {metric}')
        add_statistics_lines(df[metric].replace([np.inf, -np.inf], np.nan).dropna())
    plt.tight_layout()
    plt.show()

    # Histograms for each admin2_name
    for admin in df['admin2_name'].unique():
        plt.figure(figsize=(12, 8))
        for i, metric in enumerate(metrics, 1):
            plt.subplot(3, 1, i)
            sns.histplot(df[df['admin2_name'] == admin][metric].replace([np.inf, -np.inf], np.nan).dropna(), bins=20, kde=True)
            plt.title(f'Histogram for {metric} - {admin}')
            add_statistics_lines(df[df['admin2_name'] == admin][metric].replace([np.inf, -np.inf], np.nan).dropna())
        plt.tight_layout()
        plt.show()

## Clustering Using Precipitation Across Years

### Data Preprocessing

In [ ]:
# Create a copy of the original DataFrame
df = combined_df.copy()

# Combine year and month into a single column in the format 'YYYY-MM'
df['year_month'] = df['year'].astype(str) + '_' + df['month'].astype(str).str.zfill(2)

# Drop the original 'year' and 'month' columns
df.drop(columns=['year', 'month'], inplace=True)

# Split the DataFrame into two datasets: one for Long Rains and one for Short Rains
df_long_rains = df[df['season'] == 'Long Rains'].copy()
df_short_rains = df[df['season'] == 'Short Rains'].copy()

# Check the resulting datasets
print("Long Rains Dataset:")
print(df_long_rains.head())

print("\nShort Rains Dataset:")
print(df_short_rains.head())

In [ ]:
# Pivot the Long Rains dataset: admin2_name as index, year as columns, and precipitation as values
df_pivot_long_rains = df_long_rains.pivot(index='admin2_name', columns='year_month', values='precipitation')

# Pivot the Short Rains dataset: admin2_name as index, year as columns, and precipitation as values
df_pivot_short_rains = df_short_rains.pivot(index='admin2_name', columns='year_month', values='precipitation')

# Check the results
print("Pivoted Long Rains Dataset:")
print(df_pivot_long_rains.head())

print("\nPivoted Short Rains Dataset:")
print(df_pivot_short_rains.head())

#### Clustering Long Rains

In [ ]:
# Normalize the precipitation data
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df_pivot_long_rains),
                         index=df_pivot_long_rains.index,  # Retain the index (admin2_name)
                         columns=df_pivot_long_rains.columns)

# Resulting dataframe after preprocessing
print(df.head())

##### Feature Vector Construction

In [ ]:
# Construct the feature vector using only the precipitation columns (2000_03, 2000_04, etc.)
feature_vector = df

# Display the first few rows of the feature vector
print(feature_vector.head())

To optimize the number of components (n_components) for Principal Component Analysis (PCA), we need to determine how many components are sufficient to retain the majority of the variance in the data. This can be done by plotting the explained variance ratio and observing when additional components contribute diminishing returns.

In [ ]:
# Apply PCA with a large number of components to see variance explained by each component
# Plot the cumulative explained variance ratio
pca_model, cumulative_variance = plot_pca_variance_explained(feature_vector)

# Calculate the cumulative explained variance ratio
print("Cumulative explained variance:", cumulative_variance)

In [ ]:
# Apply PCA for dimensionality reduction
n_components = 15
pca = PCA(n_components=n_components)
reduced_features = pca.fit_transform(feature_vector)

##### K-Means Clustering

The Elbow Method is used to determine the optimal number of clusters (n_clusters) in K-Means clustering. It involves fitting the K-Means algorithm with different values for n_clusters and plotting the Within-Cluster Sum of Squares (WCSS) to observe the point where the WCSS starts to diminish at a reduced rate (forming an "elbow").

In [ ]:
# Calculate WCSS for different values of n_clusters
wcss = plot_elbow_method(reduced_features, cluster_range=range(1, 11))
print("WCSS for each number of clusters:", wcss)

In [ ]:
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(reduced_features)

# Add the cluster labels to the original dataframe
df['cluster'] = kmeans.labels_

# Display the first few rows of the dataframe with the cluster assignments
print(df.head())

In [ ]:
# Create the GeoDataFrame by merging the drought events DataFrame with the geospatial data
drought_gdf = admin_boundaries.merge(df, left_on='ADM2_NAME', right_on='admin2_name')

# Call the function to plot all K-Means clusters with a legend
plot_all_geospatial_kmeans_clusters_with_legend("Long Rains")

##### Hierarchical Clustering

Ward’s method minimizes variance and helps in identifying clusters of similar rainfall amounts. It provides a dendrogram, which can help determine different levels of clustering.

In [ ]:
# Perform hierarchical clustering using Ward's method and plot the dendrogram to visualize cluster relationships
linkage_matrix = plot_hierarchical_dendrogram(reduced_features, cut_off=5, title='Dendrogram for Clustering with Ward\'s Method')

In [ ]:
# Perform hierarchical clustering using Ward's method
linkage_matrix = linkage(reduced_features, method='ward')

# Assign clusters
n_clusters = 3
hierarchical_labels = fcluster(linkage_matrix, n_clusters, criterion='maxclust')

# Add the hierarchical cluster labels to the original dataframe
df['hierarchical_cluster'] = hierarchical_labels

In [ ]:
# Create the GeoDataFrame by merging the drought events DataFrame with the geospatial data
drought_gdf = admin_boundaries.merge(df, left_on='ADM2_NAME', right_on='admin2_name')

# Call the function to plot all hierarchical clusters with a legend
plot_all_hierarchical_clusters_with_legend("Long Rains")

##### DBSCAN Clustering

In [ ]:
# Assuming 'df' is your original DataFrame and 'reduced_features' contains the PCA-reduced data
df_with_clusters, best_eps, best_min_samples, best_score = optimize_dbscan(df, reduced_features)

# You can check the optimized results
print(f"Best eps: {best_eps}, Best min_samples: {best_min_samples}, Best Silhouette Score: {best_score}")

#### Clustering Short Rains

In [ ]:
# Normalize the precipitation data
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df_pivot_short_rains),
                         index=df_pivot_short_rains.index,  # Retain the index (admin2_name)
                         columns=df_pivot_short_rains.columns)

# Resulting dataframe after preprocessing
print(df.head())

##### Feature Vector Construction

In [ ]:
# Construct the feature vector using only the precipitation columns (2000_03, 2000_04, etc.)
feature_vector = df

# Display the first few rows of the feature vector
print(feature_vector.head())

To optimize the number of components (n_components) for Principal Component Analysis (PCA), we need to determine how many components are sufficient to retain the majority of the variance in the data. This can be done by plotting the explained variance ratio and observing when additional components contribute diminishing returns.

In [ ]:
# Apply PCA with a large number of components to see variance explained by each component
# Plot the cumulative explained variance ratio
pca_model, cumulative_variance = plot_pca_variance_explained(feature_vector)

# Calculate the cumulative explained variance ratio
print("Cumulative explained variance:", cumulative_variance)

In [ ]:
# Apply PCA for dimensionality reduction
n_components = 10
pca = PCA(n_components=n_components)
reduced_features = pca.fit_transform(feature_vector)

##### K-Means Clustering

The Elbow Method is used to determine the optimal number of clusters (n_clusters) in K-Means clustering. It involves fitting the K-Means algorithm with different values for n_clusters and plotting the Within-Cluster Sum of Squares (WCSS) to observe the point where the WCSS starts to diminish at a reduced rate (forming an "elbow").

In [ ]:
# Calculate WCSS for different values of n_clusters
wcss = plot_elbow_method(reduced_features, cluster_range=range(1, 11))
print("WCSS for each number of clusters:", wcss)

In [ ]:
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(reduced_features)

# Add the cluster labels to the original dataframe
df['cluster'] = kmeans.labels_

# Display the first few rows of the dataframe with the cluster assignments
print(df.head())

In [ ]:
# Create the GeoDataFrame by merging the drought events DataFrame with the geospatial data
drought_gdf = admin_boundaries.merge(df, left_on='ADM2_NAME', right_on='admin2_name')

# Call the function to plot all K-Means clusters with a legend
plot_all_geospatial_kmeans_clusters_with_legend("Short Rains")

##### Hierarchical Clustering

Ward’s method minimizes variance and helps in identifying clusters of similar rainfall amounts. It provides a dendrogram, which can help determine different levels of clustering.

In [ ]:
# Perform hierarchical clustering using Ward's method and plot the dendrogram to visualize cluster relationships
linkage_matrix = plot_hierarchical_dendrogram(reduced_features, cut_off=5, title='Dendrogram for Clustering with Ward\'s Method')

In [ ]:
# Perform hierarchical clustering using Ward's method
linkage_matrix = linkage(reduced_features, method='ward')

# Assign clusters
n_clusters = 3
hierarchical_labels = fcluster(linkage_matrix, n_clusters, criterion='maxclust')

# Add the hierarchical cluster labels to the original dataframe
df['hierarchical_cluster'] = hierarchical_labels

In [ ]:
# Create the GeoDataFrame by merging the drought events DataFrame with the geospatial data
drought_gdf = admin_boundaries.merge(df, left_on='ADM2_NAME', right_on='admin2_name')

# Call the function to plot all hierarchical clusters with a legend
plot_all_hierarchical_clusters_with_legend("Short Rains")

##### DBSCAN Clustering

In [ ]:
# Assuming 'df' is your original DataFrame and 'reduced_features' contains the PCA-reduced data
df_with_clusters, best_eps, best_min_samples, best_score = optimize_dbscan(df, reduced_features)

# You can check the optimized results
print(f"Best eps: {best_eps}, Best min_samples: {best_min_samples}, Best Silhouette Score: {best_score}")

## Clustering Using Drought Identification

In [ ]:
# Create a copy of the dataset to avoid modifying the original data
df = drought_events_df.copy()

# Drop the 'Drought Event' and 'region' columns
df.drop(columns=['Drought Event', 'region'], inplace=True)

# Check the result
print(df.head())

In [ ]:
# Define the unique values for 'spi_scale' and 'season'
spi_scales = df['spi_scale'].unique()
seasons = df['season'].unique()

In [ ]:
# Initialize an empty dictionary to store the datasets
datasets = {}

# Loop through each combination of 'spi_scale' and 'season'
for spi_scale in spi_scales:
    for season in seasons:
        # Create a filtered dataset for the specific combination of 'spi_scale' and 'season'
        dataset_name = f"{spi_scale}_{season}".replace(" ", "_").lower()

        # Filter the DataFrame for the current 'spi_scale' and 'season'
        df_filtered = df[(df['spi_scale'] == spi_scale) & (df['season'] == season)].copy()

        # Drop the 'spi_scale' and 'season' columns in the resulting dataset
        df_filtered.drop(columns=['spi_scale', 'season'], inplace=True)

        # Add the resulting DataFrame to the dictionary
        datasets[dataset_name] = df_filtered

In [ ]:
# Loop through the dictionary to view the index (keys) and key-value pairs
for idx, (key, value) in enumerate(datasets.items()):
    print(f"Index {idx}: Key = {key}, Dataset Shape = {value.shape}")
    print("-" * 40)

### Clustering Long Rains with SPI1

In [ ]:
# Access the first dataset
df = process_drought_dataset_by_index(datasets=datasets, index=0)

# Check the result
print(df.head())

In [ ]:
# Plots histograms for Drought Duration, Drought Severity, and Drought Intensity
plot_drought_metrics_histograms_with_stats(df)

In [ ]:
# Drought Duration - bimodal with two peaks
# Drought Severity - left-skewed (negatively skewed) distribution
# Drought Intensity -  somewhat bimodal but left skewed
df_combined = process_drought_data(df, duration_stat='median', severity_stat='median', intensity_stat='mean')

# Display the result
print(df_combined.head())

#### Data Preprocessing

In [ ]:
# Normalize the combined dataset
scaler = MinMaxScaler()

# Keep 'admin2_name' aside
admin2_names = df_combined['admin2_name']

# Exclude 'admin2_name' from the features to be scaled
df_combined_for_clustering = df_combined.drop(columns=['admin2_name'])

# Apply MinMaxScaler to the remaining features (excluding 'admin2_name')
df = pd.DataFrame(scaler.fit_transform(df_combined_for_clustering),
                         columns=df_combined_for_clustering.columns)

#### Feature Vector Construction

To optimize the number of components (n_components) for Principal Component Analysis (PCA), we need to determine how many components are sufficient to retain the majority of the variance in the data. This can be done by plotting the explained variance ratio and observing when additional components contribute diminishing returns.

In [ ]:
# Apply PCA with a large number of components to see variance explained by each component
# Plot the cumulative explained variance ratio
pca_model, cumulative_variance = plot_pca_variance_explained(df)

# Calculate the cumulative explained variance ratio
print("Cumulative explained variance:", cumulative_variance)

In [ ]:
# Choose the optimal number of components based on the plot
optimal_n_components = 17  # This value should be adjusted based on the plot

# Apply PCA with the optimal number of components
pca_optimized = PCA(n_components=optimal_n_components)
reduced_features = pca_optimized.fit_transform(df)

# Print the shape of the reduced feature set
print(f"Reduced Feature Set Shape: {reduced_features.shape}")
print("Explained Variance by Optimal Components:")
print(np.sum(pca_optimized.explained_variance_ratio_))

#### K-Means Clustering

The Elbow Method is used to determine the optimal number of clusters (n_clusters) in K-Means clustering. It involves fitting the K-Means algorithm with different values for n_clusters and plotting the Within-Cluster Sum of Squares (WCSS) to observe the point where the WCSS starts to diminish at a reduced rate (forming an "elbow").

In [ ]:
# Calculate WCSS for different values of n_clusters
wcss = plot_elbow_method(reduced_features, cluster_range=range(1, 11))
print("WCSS for each number of clusters:", wcss)

In [ ]:
# Define the number of clusters (determined using the Elbow Method or chosen directly)
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(reduced_features)

# Add the cluster labels to the original DataFrame
df['cluster'] = kmeans.labels_

# Display the first few rows of the DataFrame with the cluster assignments
print("DataFrame with K-Means cluster assignments:")
print(df.head())

In [ ]:
# Visualization of K-Means Clustering Results

# Reattach 'admin2_name' to the PCA-transformed DataFrame
df['admin2_name'] = admin2_names.values

# Create the GeoDataFrame by merging the drought metrics DataFrame with the geospatial data
drought_gdf = admin_boundaries.merge(df, left_on='ADM2_NAME', right_on='admin2_name')

# Call the function to plot all K-Means clusters with a legend
plot_all_geospatial_kmeans_clusters_with_legend("Long Rains with SPI1")

#### Hierarchical Clustering

Ward’s method minimizes variance and helps in identifying clusters of similar rainfall amounts. It provides a dendrogram, which can help determine different levels of clustering.

In [ ]:
# Perform hierarchical clustering using Ward's method and plot the dendrogram to visualize cluster relationships
linkage_matrix = plot_hierarchical_dendrogram(reduced_features, cut_off=4, title='Dendrogram for Clustering with Ward\'s Method')

In [ ]:
# Choose the Optimal Number of Clusters and Assign Labels
n_clusters = 4
hierarchical_labels = fcluster(linkage_matrix, n_clusters, criterion='maxclust')

# 4. Add the Hierarchical Cluster Labels to the Original DataFrame

df['hierarchical_cluster'] = hierarchical_labels

# Display the first few rows of the DataFrame with the cluster assignments
print("DataFrame with Hierarchical cluster assignments:")
print(df[['admin2_name', 'hierarchical_cluster']].head())

In [ ]:
# Create the GeoDataFrame by merging the drought metrics DataFrame with the geospatial data
drought_gdf = admin_boundaries.merge(df, left_on='ADM2_NAME', right_on='admin2_name')

# Call the function to plot all hierarchical clusters with a legend
plot_all_hierarchical_clusters_with_legend("Long Rains with SPI1")

#### DBSCAN Clustering

In [ ]:
# Assuming 'df' is your original DataFrame and 'reduced_features' contains the PCA-reduced data
df_with_clusters, best_eps, best_min_samples, best_score = optimize_dbscan(df, reduced_features)

# You can check the optimized results
print(f"Best eps: {best_eps}, Best min_samples: {best_min_samples}, Best Silhouette Score: {best_score}")

In [ ]:
#print(f'Number of clusters - {len(df["dbscan_cluster"].unique())}')

### Clustering Short Rains with SPI1

In [ ]:
# Access the first dataset
df = process_drought_dataset_by_index(datasets=datasets, index=1)

# Check the result
print(df.head())

In [ ]:
# Plots histograms for Drought Duration, Drought Severity, and Drought Intensity
plot_drought_metrics_histograms_with_stats(df)

In [ ]:
# Drought Duration - bimodal distribution
# Drought Severity - left-skewed (negatively skewed) distribution
# Drought Intensity -  fairly normal distribution with a small amount of skewness
df_combined = process_drought_data(df, duration_stat='mode', severity_stat='median', intensity_stat='mean')

# Display the result
print(df_combined.head())

#### Data Preprocessing

In [ ]:
# Normalize the combined dataset
scaler = MinMaxScaler()

# Keep 'admin2_name' aside
admin2_names = df_combined['admin2_name']

# Exclude 'admin2_name' from the features to be scaled
df_combined_for_clustering = df_combined.drop(columns=['admin2_name'])

# Apply MinMaxScaler to the remaining features (excluding 'admin2_name')
df = pd.DataFrame(scaler.fit_transform(df_combined_for_clustering),
                         columns=df_combined_for_clustering.columns)

#### Feature Vector Construction

To optimize the number of components (n_components) for Principal Component Analysis (PCA), we need to determine how many components are sufficient to retain the majority of the variance in the data. This can be done by plotting the explained variance ratio and observing when additional components contribute diminishing returns.

In [ ]:
# Apply PCA with a large number of components to see variance explained by each component
# Plot the cumulative explained variance ratio
pca_model, cumulative_variance = plot_pca_variance_explained(df)

# Calculate the cumulative explained variance ratio
print("Cumulative explained variance:", cumulative_variance)

In [ ]:
# Choose the optimal number of components based on the plot
optimal_n_components = 11  # This value should be adjusted based on the plot

# Apply PCA with the optimal number of components
pca_optimized = PCA(n_components=optimal_n_components)
reduced_features = pca_optimized.fit_transform(df)

# Print the shape of the reduced feature set
print(f"Reduced Feature Set Shape: {reduced_features.shape}")
print("Explained Variance by Optimal Components:")
print(np.sum(pca_optimized.explained_variance_ratio_))

#### K-Means Clustering

The Elbow Method is used to determine the optimal number of clusters (n_clusters) in K-Means clustering. It involves fitting the K-Means algorithm with different values for n_clusters and plotting the Within-Cluster Sum of Squares (WCSS) to observe the point where the WCSS starts to diminish at a reduced rate (forming an "elbow").

In [ ]:
# Calculate WCSS for different values of n_clusters
wcss = plot_elbow_method(reduced_features, cluster_range=range(1, 11))
print("WCSS for each number of clusters:", wcss)

In [ ]:
# Define the number of clusters (determined using the Elbow Method or chosen directly)
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(reduced_features)

# Add the cluster labels to the original DataFrame
df['cluster'] = kmeans.labels_

# Display the first few rows of the DataFrame with the cluster assignments
print("DataFrame with K-Means cluster assignments:")
print(df.head())

In [ ]:
# Visualization of K-Means Clustering Results

# Reattach 'admin2_name' to the PCA-transformed DataFrame
df['admin2_name'] = admin2_names.values

# Create the GeoDataFrame by merging the drought metrics DataFrame with the geospatial data
drought_gdf = admin_boundaries.merge(df, left_on='ADM2_NAME', right_on='admin2_name')

# Call the function to plot all K-Means clusters with a legend
plot_all_geospatial_kmeans_clusters_with_legend("Short Rains with SPI1")

#### Hierarchical Clustering

Ward’s method minimizes variance and helps in identifying clusters of similar rainfall amounts. It provides a dendrogram, which can help determine different levels of clustering.

In [ ]:
# Perform hierarchical clustering using Ward's method and plot the dendrogram to visualize cluster relationships
linkage_matrix = plot_hierarchical_dendrogram(reduced_features, cut_off=3, title='Dendrogram for Clustering with Ward\'s Method')

In [ ]:
# Choose the Optimal Number of Clusters and Assign Labels
n_clusters = 4
hierarchical_labels = fcluster(linkage_matrix, n_clusters, criterion='maxclust')

# 4. Add the Hierarchical Cluster Labels to the Original DataFrame

df['hierarchical_cluster'] = hierarchical_labels

# Display the first few rows of the DataFrame with the cluster assignments
print("DataFrame with Hierarchical cluster assignments:")
print(df[['admin2_name', 'hierarchical_cluster']].head())

In [ ]:
# Create the GeoDataFrame by merging the drought metrics DataFrame with the geospatial data
drought_gdf = admin_boundaries.merge(df, left_on='ADM2_NAME', right_on='admin2_name')

# Call the function to plot all hierarchical clusters with a legend
plot_all_hierarchical_clusters_with_legend("Short Rains with SPI1")

#### DBSCAN Clustering

In [ ]:
# Assuming 'df' is your original DataFrame and 'reduced_features' contains the PCA-reduced data
df_with_clusters, best_eps, best_min_samples, best_score = optimize_dbscan(df, reduced_features)

# You can check the optimized results
print(f"Best eps: {best_eps}, Best min_samples: {best_min_samples}, Best Silhouette Score: {best_score}")

In [ ]:
#print(f'Number of clusters - {len(df["dbscan_cluster"].unique())}')

### Clustering Long Rains with SPI3

In [ ]:
# Access the first dataset
df = process_drought_dataset_by_index(datasets=datasets, index=2)

# Check the result
print(df.head())

In [ ]:
# Plots histograms for Drought Duration, Drought Severity, and Drought Intensity
plot_drought_metrics_histograms_with_stats(df)

In [ ]:
# Drought Duration - right-skewed distribution
# Drought Severity - left-skewed (negatively skewed) distribution
# Drought Intensity -  somewhat bimodal, with peaks
df_combined = process_drought_data(df, duration_stat='median', severity_stat='median', intensity_stat='mean')

# Display the result
print(df_combined.head())

#### Data Preprocessing

In [ ]:
# Normalize the combined dataset
scaler = MinMaxScaler()

# Keep 'admin2_name' aside
admin2_names = df_combined['admin2_name']

# Exclude 'admin2_name' from the features to be scaled
df_combined_for_clustering = df_combined.drop(columns=['admin2_name'])

# Apply MinMaxScaler to the remaining features (excluding 'admin2_name')
df = pd.DataFrame(scaler.fit_transform(df_combined_for_clustering),
                         columns=df_combined_for_clustering.columns)

#### Feature Vector Construction

To optimize the number of components (n_components) for Principal Component Analysis (PCA), we need to determine how many components are sufficient to retain the majority of the variance in the data. This can be done by plotting the explained variance ratio and observing when additional components contribute diminishing returns.

In [ ]:
# Apply PCA with a large number of components to see variance explained by each component
# Plot the cumulative explained variance ratio
pca_model, cumulative_variance = plot_pca_variance_explained(df)

# Calculate the cumulative explained variance ratio
print("Cumulative explained variance:", cumulative_variance)

In [ ]:
# Choose the optimal number of components based on the plot
optimal_n_components = 20  # This value should be adjusted based on the plot

# Apply PCA with the optimal number of components
pca_optimized = PCA(n_components=optimal_n_components)
reduced_features = pca_optimized.fit_transform(df)

# Print the shape of the reduced feature set
print(f"Reduced Feature Set Shape: {reduced_features.shape}")
print("Explained Variance by Optimal Components:")
print(np.sum(pca_optimized.explained_variance_ratio_))

#### K-Means Clustering

The Elbow Method is used to determine the optimal number of clusters (n_clusters) in K-Means clustering. It involves fitting the K-Means algorithm with different values for n_clusters and plotting the Within-Cluster Sum of Squares (WCSS) to observe the point where the WCSS starts to diminish at a reduced rate (forming an "elbow").

In [ ]:
# Calculate WCSS for different values of n_clusters
wcss = plot_elbow_method(reduced_features, cluster_range=range(1, 11))
print("WCSS for each number of clusters:", wcss)

In [ ]:
# Define the number of clusters (determined using the Elbow Method or chosen directly)
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(reduced_features)

# Add the cluster labels to the original DataFrame
df['cluster'] = kmeans.labels_

# Display the first few rows of the DataFrame with the cluster assignments
print("DataFrame with K-Means cluster assignments:")
print(df.head())

In [ ]:
# Visualization of K-Means Clustering Results

# Reattach 'admin2_name' to the PCA-transformed DataFrame
df['admin2_name'] = admin2_names.values

# Create the GeoDataFrame by merging the drought metrics DataFrame with the geospatial data
drought_gdf = admin_boundaries.merge(df, left_on='ADM2_NAME', right_on='admin2_name')

# Call the function to plot all K-Means clusters with a legend
plot_all_geospatial_kmeans_clusters_with_legend("Long Rains with SPI3")

#### Hierarchical Clustering

Ward’s method minimizes variance and helps in identifying clusters of similar rainfall amounts. It provides a dendrogram, which can help determine different levels of clustering.

In [ ]:
# Perform hierarchical clustering using Ward's method and plot the dendrogram to visualize cluster relationships
linkage_matrix = plot_hierarchical_dendrogram(reduced_features, cut_off=4, title='Dendrogram for Clustering with Ward\'s Method')

In [ ]:
# Choose the Optimal Number of Clusters and Assign Labels

n_clusters = 5
hierarchical_labels = fcluster(linkage_matrix, n_clusters, criterion='maxclust')

# 4. Add the Hierarchical Cluster Labels to the Original DataFrame

df['hierarchical_cluster'] = hierarchical_labels

# Display the first few rows of the DataFrame with the cluster assignments
print("DataFrame with Hierarchical cluster assignments:")
print(df[['admin2_name', 'hierarchical_cluster']].head())

In [ ]:
# Create the GeoDataFrame by merging the drought metrics DataFrame with the geospatial data
drought_gdf = admin_boundaries.merge(df, left_on='ADM2_NAME', right_on='admin2_name')

# Call the function to plot all hierarchical clusters with a legend
plot_all_hierarchical_clusters_with_legend("Long Rains with SPI3")

#### DBSCAN Clustering

In [ ]:
# Assuming 'df' is your original DataFrame and 'reduced_features' contains the PCA-reduced data
df_with_clusters, best_eps, best_min_samples, best_score = optimize_dbscan(df, reduced_features)

# You can check the optimized results
print(f"Best eps: {best_eps}, Best min_samples: {best_min_samples}, Best Silhouette Score: {best_score}")

In [ ]:
#print(f'Number of clusters - {len(df["dbscan_cluster"].unique())}')

### Clustering Short Rains with SPI3

In [ ]:
# Access the first dataset
df = process_drought_dataset_by_index(datasets=datasets, index=3)

# Check the result
print(df.head())

In [ ]:
# Plots histograms for Drought Duration, Drought Severity, and Drought Intensity
plot_drought_metrics_histograms_with_stats(df)

In [ ]:
# Drought Duration -  bimodal distribution
# Drought Severity - left-skewed (negatively skewed) distribution
# Drought Intensity -  somewhat bimodal but left skewed
df_combined = process_drought_data(df, duration_stat='median', severity_stat='median', intensity_stat='median')


# Display the result
print(df_combined.head())

#### Data Preprocessing

In [ ]:
# Normalize the combined dataset
scaler = MinMaxScaler()

# Keep 'admin2_name' aside
admin2_names = df_combined['admin2_name']

# Exclude 'admin2_name' from the features to be scaled
df_combined_for_clustering = df_combined.drop(columns=['admin2_name'])

# Apply MinMaxScaler to the remaining features (excluding 'admin2_name')
df = pd.DataFrame(scaler.fit_transform(df_combined_for_clustering),
                         columns=df_combined_for_clustering.columns)

#### Feature Vector Construction

To optimize the number of components (n_components) for Principal Component Analysis (PCA), we need to determine how many components are sufficient to retain the majority of the variance in the data. This can be done by plotting the explained variance ratio and observing when additional components contribute diminishing returns.

In [ ]:
# Apply PCA with a large number of components to see variance explained by each component
# Plot the cumulative explained variance ratio
pca_model, cumulative_variance = plot_pca_variance_explained(df)

# Calculate the cumulative explained variance ratio
print("Cumulative explained variance:", cumulative_variance)

In [ ]:
# Choose the optimal number of components based on the plot
optimal_n_components = 10  # This value should be adjusted based on the plot

# Apply PCA with the optimal number of components
pca_optimized = PCA(n_components=optimal_n_components)
reduced_features = pca_optimized.fit_transform(df)

# Print the shape of the reduced feature set
print(f"Reduced Feature Set Shape: {reduced_features.shape}")
print("Explained Variance by Optimal Components:")
print(np.sum(pca_optimized.explained_variance_ratio_))

#### K-Means Clustering

The Elbow Method is used to determine the optimal number of clusters (n_clusters) in K-Means clustering. It involves fitting the K-Means algorithm with different values for n_clusters and plotting the Within-Cluster Sum of Squares (WCSS) to observe the point where the WCSS starts to diminish at a reduced rate (forming an "elbow").

In [ ]:
# Calculate WCSS for different values of n_clusters
wcss = plot_elbow_method(reduced_features, cluster_range=range(1, 11))
print("WCSS for each number of clusters:", wcss)

In [ ]:
# Define the number of clusters (determined using the Elbow Method or chosen directly)
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(reduced_features)

# Add the cluster labels to the original DataFrame
df['cluster'] = kmeans.labels_

# Display the first few rows of the DataFrame with the cluster assignments
print("DataFrame with K-Means cluster assignments:")
print(df.head())

In [ ]:
# Visualization of K-Means Clustering Results

# Reattach 'admin2_name' to the PCA-transformed DataFrame
df['admin2_name'] = admin2_names.values

# Create the GeoDataFrame by merging the drought metrics DataFrame with the geospatial data
drought_gdf = admin_boundaries.merge(df, left_on='ADM2_NAME', right_on='admin2_name')

# Call the function to plot all K-Means clusters with a legend
plot_all_geospatial_kmeans_clusters_with_legend("Short Rains with SPI3")

#### Hierarchical Clustering

Ward’s method minimizes variance and helps in identifying clusters of similar rainfall amounts. It provides a dendrogram, which can help determine different levels of clustering.

In [ ]:
# Perform hierarchical clustering using Ward's method and plot the dendrogram to visualize cluster relationships
linkage_matrix = plot_hierarchical_dendrogram(reduced_features, cut_off=3, title='Dendrogram for Clustering with Ward\'s Method')

In [ ]:
# Choose the Optimal Number of Clusters and Assign Labels

n_clusters = 4
hierarchical_labels = fcluster(linkage_matrix, n_clusters, criterion='maxclust')

# 4. Add the Hierarchical Cluster Labels to the Original DataFrame

df['hierarchical_cluster'] = hierarchical_labels

# Display the first few rows of the DataFrame with the cluster assignments
print("DataFrame with Hierarchical cluster assignments:")
print(df[['admin2_name', 'hierarchical_cluster']].head())

In [ ]:
# Create the GeoDataFrame by merging the drought metrics DataFrame with the geospatial data
drought_gdf = admin_boundaries.merge(df, left_on='ADM2_NAME', right_on='admin2_name')

# Call the function to plot all hierarchical clusters with a legend
plot_all_hierarchical_clusters_with_legend("Short Rains with SPI3")

#### DBSCAN Clustering

In [ ]:
# Assuming 'df' is your original DataFrame and 'reduced_features' contains the PCA-reduced data
df_with_clusters, best_eps, best_min_samples, best_score = optimize_dbscan(df, reduced_features)

# You can check the optimized results
print(f"Best eps: {best_eps}, Best min_samples: {best_min_samples}, Best Silhouette Score: {best_score}")

In [ ]:
#print(f'Number of clusters - {len(df["dbscan_cluster"].unique())}')

# EM-DAT Analysis

In [ ]:
pip install openpyxl

In [ ]:
# Load the Excel file
file_path = 'public_emdat_custom_request_2024-10-01_36a9efba-9545-41bc-a272-3b49c638962b.xlsx'
excel_data = pd.ExcelFile(file_path)

# Display sheet names to understand the structure
sheet_names = excel_data.sheet_names

# Load the data from each sheet
emdat_data = pd.read_excel(excel_data, sheet_name='EM-DAT Data')
emdat_info = pd.read_excel(excel_data, sheet_name='EM-DAT Info')

# Clean the 'EM-DAT Data' sheet by dropping unnecessary columns and handling missing values
# Drop columns that seem unnecessary for analysis
columns_to_drop = [
    'External IDs', 'Event Name', 'Reconstruction Costs (\'000 US$)', 'Classification Key',
    'Reconstruction Costs, Adjusted (\'000 US$)', 'Insured Damage (\'000 US$)',
    'Insured Damage, Adjusted (\'000 US$)', 'Total Damage (\'000 US$)', 'No. Injured',
    'Total Damage, Adjusted (\'000 US$)', 'Entry Date', 'Last Update', 'No. Homeless', 'Disaster Group',
    'Disaster Subgroup', 'Disaster Type', 'Disaster Subtype', 'ISO', 'Country', 'Subregion', 'Region',
    'OFDA/BHA Response', 'Appeal', 'Declaration', "AID Contribution ('000 US$)",
    'Magnitude', 'Magnitude Scale', 'Latitude',	'Longitude', 'River Basin', 'Start Day', 'End Day'
]

emdat_data = emdat_data.drop(columns=columns_to_drop)

# Reset the index for a clean DataFrame
emdat_data.reset_index(drop=True, inplace=True)

# Separate the values in the 'Location' column into multiple columns

# Split the 'Location' column into multiple columns
location_split = emdat_data['Location'].str.split(',', expand=True)

# Rename the new columns as 'Location_1', 'Location_2', etc.
location_split.columns = [f'Location_{i+1}' for i in range(location_split.shape[1])]

# Concatenate the original dataframe with the new location columns
emdat_data = pd.concat([emdat_data.drop(columns=['Location']), location_split], axis=1)

# Separate the values in the 'Admin Units' column into multiple columns
# Assuming 'Admin Units' is in JSON format; parsing and normalizing it
admin_units_split = emdat_data['Admin Units'].apply(lambda x: pd.json_normalize(eval(x)) if pd.notna(x) else pd.DataFrame())

# Concatenate the parsed 'Admin Units' data into a single DataFrame
admin_units_expanded = pd.concat(admin_units_split.tolist(), ignore_index=True)

# Rename the columns appropriately as 'Admin_Unit_1', 'Admin_Unit_2', etc.
admin_units_expanded.columns = [f'Admin_Unit_{col}' for col in admin_units_expanded.columns]

# Concatenate the original dataframe with the new admin unit columns
emdat_data = pd.concat([emdat_data.drop(columns=['Admin Units']), admin_units_expanded], axis=1)

# Melt the dataset to create a 'Location' column from the separated location columns
# Identifying the id_vars (columns that should remain fixed) and value_vars (columns to unpivot)
id_vars = [col for col in emdat_data.columns if not col.startswith('Location_')]
value_vars = [col for col in emdat_data.columns if col.startswith('Location_')]

# Melting the dataset
emdat_data = pd.melt(
    emdat_data,
    id_vars=id_vars,
    value_vars=value_vars,
    var_name='Location_Number',
    value_name='Location'
)

# Drop rows where 'Location' is NaN (since these are redundant after melting)
emdat_data = emdat_data.dropna(subset=['Location']).reset_index(drop=True)

# Strip the white space from the 'Location' column
emdat_data['Location'] = emdat_data['Location'].str.strip()

# Drop the 'Location_Number' column
emdat_data = emdat_data.drop(columns=['Location_Number', 'Admin_Unit_adm1_code', 'Admin_Unit_adm2_code'])

# Creating a dictionary mapping for the locations to their corresponding admin 1 regions
location_to_admin1_dict = {
    'Kilifi': 'Coast',
    'Kwale': 'Coast',
    'Malindi': 'Coast',
    'Taita Taveta': 'Coast',
    'Tana River': 'Coast',
    'Kitui': 'Eastern',
    'Marsabit': 'Eastern'
}

# Map the 'Admin_Unit_adm2_name' to 'Admin_Unit_adm1_name' using the dictionary
emdat_data['Admin_Unit_adm1_name'] = emdat_data['Admin_Unit_adm2_name'].map(location_to_admin1_dict)

# Drop rows where the value of "Admin_Unit_adm1_name" is "Coast"
emdat_data = emdat_data[emdat_data['Admin_Unit_adm1_name'] != 'Coast'].reset_index(drop=True)

# Display the updated DataFrame with the mapped admin 1 regions
emdat_data

In [ ]:
# Get the unique pairings between "Start Year" and "End Year"
unique_year_pairings = emdat_data[['Start Year', 'End Year']].dropna().drop_duplicates().reset_index(drop=True)

# Display the unique pairings
unique_year_pairings